In [ ]:
############ combination of solution to solve more #########

Acknowledgment:
This notebook combines my original work with adapted components and ideas inspired by  public Kaggle notebooks output solution:

https://www.kaggle.com/code/bibanh/qwen2-5-32b-arc-local-score


/kaggle/input/solved-127-problems-local-pleaseupvote

https://www.kaggle.com/code/jacekwl/a-bit-more-of-code-golf


I have restructured and integrated the techniques into my own pipeline, with modifications to logic, patterns, and implementation. Much respect to both authors for their contributions Use solution here combined.

In [ ]:
import json
import os
import zipfile
import numpy as np
from collections import defaultdict
from scipy.ndimage import label, binary_dilation, binary_erosion
from sklearn.cluster import KMeans


class ARCSolutionGenerator:
    def __init__(self):
        self.pattern_handlers = [
             self.handle_empty_input,          # New - handles empty grids
            self.handle_boolean_operations,   # New - fixes Tasks 059-064
            
            self.handle_pad_to_size,
            self.handle_crop_to_size,
            self.handle_scale_objects,        
            self.handle_upscale_nearest,
            self.handle_downscale_sample,

            # Basic shape operations
            self.handle_outline_only,          
            self.handle_center_object,         
            self.handle_diagonal_mirror,       
            self.handle_color_palette_row,     
            self.handle_bounding_fill_by_color,
            self.handle_row_col_propagation,   
            
            # Common pattern handlers
            self.handle_repeat_pattern,        
            self.handle_checkerboard,          
            self.handle_vertical_stripes,      
            self.handle_horizontal_stripes,    
            self.handle_majority_color_fill,   
            self.handle_cross_lines,           
            self.handle_color_swap,            
            self.handle_crop_center,           
            self.handle_single_color_output,   
            self.handle_bounding_crop,         
            self.handle_overlay_fill,          
            self.handle_remove_color,          
            
            # Color operations
            self.handle_color_mapping,
            self.handle_color_shift_plus_one,
            self.handle_keep_colors,
            
            # Grid operations
            self.handle_grid_operations,
            self.handle_object_operations,
            self.handle_pattern_replication,
            self.handle_mirror_symmetry,
            
            # Arithmetic operations
            self.handle_arithmetic_operations,
            self.handle_conditional_operations,
            
            # Advanced pattern recognition
            self.handle_upscale_nearest,            
            self.handle_downscale_sample,           
            self.handle_fill_gaps_rowwise,          
            self.handle_background_to_majority,     
            self.handle_keep_dominant_color,        
            self.handle_extract_first_nonempty_rowcol,
            self.handle_sort_rows_by_density,       
            self.handle_sort_cols_by_density,       
            self.handle_complete_by_mirror_half,    
            self.handle_palette_column,             
            
            # Newly added specialized handlers
            self.handle_uniform_row_fill,       
            self.handle_uniform_col_fill,       
            self.handle_draw_frame,             
            self.handle_main_diag_line,         
            self.handle_anti_diag_line,         
            self.handle_border_to_zero,         
            self.handle_single_object_translate,
            self.handle_object_count_row,       
            self.handle_remove_small_objects,   
            
            # Rotation and flipping handlers
            self.handle_rotate_90,
            self.handle_rotate_180,
            self.handle_rotate_270,
            self.handle_flip_horizontal,
            self.handle_flip_vertical,
            
            # New color transformation handlers
            self.handle_invert_colors,
            self.handle_shift_colors,
            self.handle_threshold_colors,
            
            # New shape transformation handlers
            self.handle_expand_borders,
            self.handle_shrink_objects,
            self.handle_connect_components,
            
            # New advanced pattern handlers
            self.handle_extract_largest_object,
            self.handle_remove_background,
            self.handle_keep_only_background,
            
            # Additional advanced handlers
            self.handle_color_gradient,
            self.handle_histogram_equalization,
            self.handle_fill_enclosed_areas,
            self.handle_remove_isolated_pixels,
            self.handle_find_contours,
            self.handle_scale_objects,
            self.handle_align_objects,
            self.handle_count_objects,
            self.handle_separate_objects,
               self.handle_fill_enclosed_areas_fixed,
        self.handle_remove_isolated_pixels_fixed
            ]

    def handle_remove_isolated_pixels_fixed(self, task_data, analysis):
        """Fixed version using proper bitwise operations"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape:
            return None
        
        # Correct implementation using bitwise operations
        isolated = (inp > 0) & ~binary_dilation(binary_erosion(inp > 0))
        result = np.where(isolated, 0, inp)
        
        if np.array_equal(out, result):
            return """def p(g):
        from scipy.ndimage import binary_dilation, binary_erosion
        import numpy as np
        arr = np.array(g)
        isolated = (arr > 0) & ~binary_dilation(binary_erosion(arr > 0))
        return np.where(isolated, 0, arr).tolist()
    """
        return None    
    def handle_grid_padding(self, task_data, analysis):
        """Handle cases where output is input with added padding"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if output is input with symmetric padding
        if (out.shape[0] > inp.shape[0] and out.shape[1] > inp.shape[1] and
            out[out.shape[0]//2 - inp.shape[0]//2 : out.shape[0]//2 + inp.shape[0]//2,
            out.shape[1]//2 - inp.shape[1]//2 : out.shape[1]//2 + inp.shape[1]//2] == inp):
            
            return f"""def p(g):
     h_pad = {out.shape[0] - inp.shape[0]}
     w_pad = {out.shape[1] - inp.shape[1]}
     return np.pad(g, ((h_pad//2, h_pad - h_pad//2),
                      (w_pad//2, w_pad - w_pad//2)), mode='constant').tolist()
    """
        return None
    
    def handle_fill_enclosed_areas_fixed(self, task_data, analysis):
        """Fixed version using proper bitwise operations"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape:
            return None
        
        # Use bitwise operations instead of subtraction
        border = binary_dilation(inp > 0) & ~binary_erosion(inp > 0)
        filled = np.where(border, inp, 0)
        
        if np.array_equal(out, filled):
            return """def p(g):
     from scipy.ndimage import binary_dilation, binary_erosion
     import numpy as np
     arr = np.array(g)
     border = binary_dilation(arr > 0) & ~binary_erosion(arr > 0)
     return np.where(border, arr, 0).tolist()
    """
        return None
    def handle_boolean_operations(self, task_data, analysis):
        """Handle boolean operations (AND/OR/XOR) between grids"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if inp.shape != out.shape:
            return None
        
        # Check for common boolean patterns
        patterns = [
            ('AND', np.bitwise_and),
            ('OR', np.bitwise_or),
            ('XOR', np.bitwise_xor),
            ('NOT', lambda a, b: ~a)
        ]
        
        for name, op in patterns:
            try:
                if name == 'NOT':
                    test = op(inp > 0)
                else:
                    test = op(inp > 0, out > 0)
                
                if np.array_equal(test, out > 0):
                    return f"""def p(g):
     import numpy as np
     arr = np.array(g)
     return np.{name.lower()}(arr > 0).astype(int).tolist()
    """
            except:
                continue
        
        return None
    
    def handle_empty_input(self, task_data, analysis):
            """Handle cases where input grid is empty"""
            inp = np.array(task_data['train'][0]['input'])
            out = np.array(task_data['train'][0]['output'])
            
            if inp.size == 0 or inp.shape[0] == 0:
                return f"""def p(g):
         return {out.tolist()}
        """
            return None    
    def handle_pad_to_size(self, task_data, analysis):
        """Pad input to match output size with zeros"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if output is input padded with zeros
        if (out.shape[0] >= inp.shape[0] and out.shape[1] >= inp.shape[1] and
            np.array_equal(out[:inp.shape[0], :inp.shape[1]], inp)):
            
            pad_top = (out.shape[0] - inp.shape[0]) // 2
            pad_left = (out.shape[1] - inp.shape[1]) // 2
            
            return f"""def p(g):
 h,w = {out.shape[0]},{out.shape[1]}
 ih,iw = len(g),len(g[0])
 res = [[0]*w for _ in range(h)]
 for i in range(ih):
  for j in range(iw):
   res[i+{pad_top}][j+{pad_left}] = g[i][j]
 return res
"""
        return None

    def handle_crop_to_size(self, task_data, analysis):
        """Crop input to match output size"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if output is a crop of input
        if (out.shape[0] <= inp.shape[0] and out.shape[1] <= inp.shape[1]):
            for y in range(inp.shape[0] - out.shape[0] + 1):
                for x in range(inp.shape[1] - out.shape[1] + 1):
                    if np.array_equal(inp[y:y+out.shape[0], x:x+out.shape[1]], out):
                        return f"""def p(g):
 return [r[{x}:{x+out.shape[1]}]] for r in g[{y}:{y+out.shape[0]}]]]
"""
        return None

    def handle_upscale_nearest(self, task_data, analysis):
        """Upscale using nearest neighbor interpolation"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check integer scaling factors
        for h_scale in range(1, 4):
            for w_scale in range(1, 4):
                if (inp.shape[0]*h_scale == out.shape[0] and 
                    inp.shape[1]*w_scale == out.shape[1]):
                    test = np.repeat(np.repeat(inp, h_scale, axis=0), w_scale, axis=1)
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return [[x for x in row for _ in range({w_scale})] 
         for row in g for _ in range({h_scale})]
"""
        return None

    def handle_downscale_sample(self, task_data, analysis):
        """Downscale by sampling pixels"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check integer scaling factors
        for h_scale in range(1, 4):
            for w_scale in range(1, 4):
                if (inp.shape[0]//h_scale == out.shape[0] and 
                    inp.shape[1]//w_scale == out.shape[1]):
                    test = inp[::h_scale, ::w_scale]
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return [row[::{w_scale}] for row in g[::{h_scale}]]
"""
        return None

    def handle_scale_objects(self, task_data, analysis):
        """Scale objects by a factor while maintaining positions"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Find connected components
        labeled, n = label(inp > 0)
        if n == 0: return None
        
        # Check if each object is scaled
        for scale in [2, 3, 0.5]:
            test = np.zeros_like(out)
            for i in range(1, n+1):
                obj = (labeled == i)
                y, x = np.where(obj)
                if len(y) == 0: continue
                
                # Calculate center
                cy, cx = np.mean(y), np.mean(x)
                
                # Scale positions
                scaled_y = ((y - cy) * scale + cy).astype(int)
                scaled_x = ((x - cx) * scale + cx).astype(int)
                
                # Place in output
                for dy, dx in zip(scaled_y, scaled_x):
                    if 0 <= dy < out.shape[0] and 0 <= dx < out.shape[1]:
                        test[dy,dx] = inp[y[0],x[0]]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 from scipy.ndimage import label
 import numpy as np
 arr = np.array(g)
 labeled, n = label(arr > 0)
 res = np.zeros({out.shape}, dtype=int)
 for i in range(1, n+1):
  obj = (labeled == i)
  y, x = np.where(obj)
  if len(y) == 0: continue
  cy, cx = np.mean(y), np.mean(x)
  scaled_y = ((y - cy) * {scale} + cy).astype(int)
  scaled_x = ((x - cx) * {scale} + cx).astype(int)
  for dy, dx in zip(scaled_y, scaled_x):
   if 0 <= dy < {out.shape[0]} and 0 <= dx < {out.shape[1]}:
    res[dy,dx] = arr[y[0],x[0]]
 return res.tolist()
"""
        return None
    
    # ----------------- Utility Methods -----------------    
    def analyze_task(self, task_data):
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        analysis = {
            'color_changes': defaultdict(set),
            'shape_changes': set(),
            'object_properties': [],
            'symmetry': None,
            'arithmetic': None,
            'color_stats': defaultdict(int)
        }
        
        for example in all_examples:
            in_grid = np.array(example['input'])
            out_grid = np.array(example['output'])
            
            analysis['shape_changes'].add((in_grid.shape, out_grid.shape))
            
            for (i,j), val in np.ndenumerate(in_grid):
                if i < out_grid.shape[0] and j < out_grid.shape[1]:
                    if in_grid[i,j] != out_grid[i,j]:
                        analysis['color_changes'][(i,j)].add((int(in_grid[i,j]), int(out_grid[i,j])))
            
            analysis['object_properties'].append(self.analyze_objects(in_grid, out_grid))
            
            if analysis['symmetry'] is None:
                analysis['symmetry'] = self.detect_symmetry(in_grid, out_grid)
            
            if analysis['arithmetic'] is None:
                analysis['arithmetic'] = self.detect_arithmetic(in_grid, out_grid)
            
            for color in in_grid.ravel():
                analysis['color_stats'][int(color)] += 1
        
        return analysis
    
    def analyze_objects(self, in_grid, out_grid):
        structure = np.ones((3,3), dtype=int)
        labeled_in, n_in = label(in_grid > 0, structure)
        labeled_out, n_out = label(out_grid > 0, structure)
        
        objects = []
        for i in range(1, n_in+1):
            in_obj = (labeled_in == i)
            out_obj = None
            if i <= n_out:
                out_obj = (labeled_out == i)
            objects.append({
                'in_size': int(in_obj.sum()),
                'out_size': int(out_obj.sum()) if out_obj is not None else 0,
                'position_change': self.detect_position_change(in_obj, out_obj),
                'color_change': self.detect_color_change(in_obj, out_obj, in_grid, out_grid)
            })
        return objects

    def detect_position_change(self, in_obj, out_obj):
        if out_obj is None:
            return "removed"
        in_pos = np.argwhere(in_obj).mean(axis=0)
        out_pos = np.argwhere(out_obj).mean(axis=0)
        return out_pos - in_pos
    
    def detect_color_change(self, in_obj, out_obj, in_grid, out_grid):
        if out_obj is None:
            return "removed"
        in_color = in_grid[in_obj][0] if np.any(in_obj) else None
        out_color = out_grid[out_obj][0] if np.any(out_obj) else None
        return (in_color, out_color) if in_color != out_color else None
    
    def detect_symmetry(self, in_grid, out_grid):
        if np.array_equal(out_grid, np.rot90(in_grid, 1)):
            return 'rotate_90'
        if np.array_equal(out_grid, np.rot90(in_grid, 2)):
            return 'rotate_180'
        if np.array_equal(out_grid, np.rot90(in_grid, 3)):
            return 'rotate_270'
        if np.array_equal(out_grid, np.flipud(in_grid)):
            return 'flip_vertical'
        if np.array_equal(out_grid, np.fliplr(in_grid)):
            return 'flip_horizontal'
        return None
    
    def detect_arithmetic(self, in_grid, out_grid):
        if in_grid.shape != out_grid.shape:
            return None
            
        diff = out_grid - in_grid
        if np.all(diff == diff[0,0]):
            return ('add', int(diff[0,0]))
        
        with np.errstate(divide='ignore', invalid='ignore'):
            safe_in = np.where(in_grid==0, 1, in_grid)
            ratio = out_grid / safe_in
            if np.all(ratio[in_grid > 0] == ratio[in_grid > 0][0]):
                return ('multiply', float(ratio[in_grid > 0][0]))
        return None
    

    
    
    # ----------------- Basic Pattern Handlers -----------------
    
    def handle_outline_only(self, task_data, analysis):
        """Keep only the outline of objects"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        edge = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i,j]==0: continue
                neighbors = [(i+1,j),(i-1,j),(i,j+1),(i,j-1)]
                if any(not (0<=y<inp.shape[0] and 0<=x<inp.shape[1]) or inp[y,x]==0 
                   for y,x in neighbors):
                    edge[i,j] = inp[i,j]
                    
        if np.array_equal(out, edge):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[[g[i][j] if g[i][j] and any((y<0 or y>=h or x<0 or x>=w or g[y][x]==0)
        for y,x in ((i+1,j),(i-1,j),(i,j+1),(i,j-1))) 
        else 0 for j in range(w)] for i in range(h)]
"""
        return None

    def handle_center_object(self, task_data, analysis):
        """Center the object in the grid"""
        inp = task_data['train'][0]['input']
        out = task_data['train'][0]['output']
        if len(inp) != len(out) or len(inp[0]) != len(out[0]): return None
        
        ys = [i for i, row in enumerate(inp) for v in row if v]
        xs = [j for row in inp for j, v in enumerate(row) if v]
        if not ys: return None
        
        y0, y1 = min(ys), max(ys) + 1
        x0, x1 = min(xs), max(xs) + 1
        crop = [r[x0:x1] for r in inp[y0:y1]]
        H, W = len(inp), len(inp[0])
        h, w = len(crop), len(crop[0])
        cy, cx = (H - h) // 2, (W - w) // 2
        
        canvas = [[0] * W for _ in range(H)]
        for i in range(h):
            canvas[cy + i][cx:cx + w] = crop[i][:]
            
        if canvas == out:
            return '''def p(g):
 H, W = len(g), len(g[0])
 ys = [i for i,row in enumerate(g) for v in row if v]
 xs = [j for row in g for j,v in enumerate(row) if v]
 y0, y1 = min(ys), max(ys)+1
 x0, x1 = min(xs), max(xs)+1
 crop = [r[x0:x1] for r in g[y0:y1]]
 h, w = len(crop), len(crop[0])
 cy, cx = (H-h)//2, (W-w)//2
 res = [[0]*W for _ in range(H)]
 for i in range(h):
     res[cy+i][cx:cx+w] = crop[i][:]
 return res
'''
        return None

    def handle_diagonal_mirror(self, task_data, analysis):
        """Mirror the grid along the main diagonal"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape[0] != out.shape[1] or inp.shape[1] != out.shape[0]: return None
        
        mirrored = inp.T
        if np.array_equal(out, mirrored):
            return """def p(g):
 return [list(r) for r in zip(*g)]
"""
        return None

    def handle_color_palette_row(self, task_data, analysis):
        """Apply a color palette from a specific row"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find a row that could serve as palette
        for row_idx in range(inp.shape[0]):
            palette_row = inp[row_idx]
            colors = {c:i for i,c in enumerate(palette_row) if c > 0}
            if not colors: continue
            
            # Try applying this palette
            test = np.zeros_like(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if inp[i,j] in colors:
                        test[i,j] = palette_row[colors[inp[i,j]]]
            
            if np.array_equal(out, test):
                return f"""def p(g):
 palette = g[{row_idx}]
 colors = {{c:i for i,c in enumerate(palette) if c>0}}
 return [[palette[colors.get(x,0)] for x in row] for row in g]
"""
        return None

    def handle_bounding_fill_by_color(self, task_data, analysis):
        """Fill bounding boxes of objects with their color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        labeled, n = label(inp > 0)
        if n == 0: return None
        
        result = np.zeros_like(inp)
        for i in range(1, n+1):
            mask = (labeled == i)
            ys, xs = np.where(mask)
            if len(ys) == 0: continue
            color = inp[ys[0], xs[0]]
            y0, y1 = min(ys), max(ys)
            x0, x1 = min(xs), max(xs)
            result[y0:y1+1, x0:x1+1] = color
            
        if np.array_equal(out, result):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 arr = np.array(g)
 labeled, n = label(arr > 0)
 res = np.zeros_like(arr)
 for i in range(1, n+1):
  mask = (labeled == i)
  ys, xs = np.where(mask)
  if len(ys) == 0: continue
  color = arr[ys[0], xs[0]]
  y0, y1 = min(ys), max(ys)
  x0, x1 = min(xs), max(xs)
  res[y0:y1+1, x0:x1+1] = color
 return res.tolist()
"""
        return None

    def handle_row_col_propagation(self, task_data, analysis):
        """Propagate row or column patterns"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check row propagation
        for row_idx in range(inp.shape[0]):
            if np.all(out == inp[row_idx:row_idx+1,:]):
                return f"""def p(g):
 return [g[{row_idx}][:] for _ in g]
"""
        
        # Check column propagation
        for col_idx in range(inp.shape[1]):
            if np.all(out == inp[:,col_idx:col_idx+1]):
                return f"""def p(g):
 return [[g[i][{col_idx}]]*len(g[0]) for i in range(len(g))]
"""
        return None

    def handle_repeat_pattern(self, task_data, analysis):
        """Repeat a smaller pattern across the grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Try different pattern sizes
        for h in range(1, min(5, inp.shape[0]+1)):
            for w in range(1, min(5, inp.shape[1]+1)):
                pattern = inp[:h,:w]
                test = np.tile(pattern, (out.shape[0]//h + 1, out.shape[1]//w + 1))
                test = test[:out.shape[0], :out.shape[1]]
                if np.array_equal(out, test):
                    return f"""def p(g):
 pattern = [r[:{w}] for r in g[:{h}]]
 return [row[:len(g[0])] for _ in range(len(g)) 
  for row in pattern * (len(g)//{h} + 1)][:len(g)]
"""
        return None

    def handle_checkerboard(self, task_data, analysis):
        """Create a checkerboard pattern"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check for alternating colors
        color1, color2 = out[0,0], out[0,1] if out.shape[1]>1 else out[1,0]
        test = np.zeros_like(out)
        for i in range(out.shape[0]):
            for j in range(out.shape[1]):
                test[i,j] = color1 if (i+j)%2==0 else color2
                
        if np.array_equal(out, test):
            return f"""def p(g):
 return [[{color1} if (i+j)%2==0 else {color2} 
  for j in range(len(g[0]))] for i in range(len(g))]
"""
        return None

    def handle_vertical_stripes(self, task_data, analysis):
        """Create vertical stripes"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find repeating column pattern
        for period in range(1, min(5, out.shape[1]+1)):
            pattern = out[:,:period]
            test = np.tile(pattern, (1, out.shape[1]//period + 1))
            test = test[:, :out.shape[1]]
            if np.array_equal(out, test):
                return f"""def p(g):
 pattern = [row[:{period}] for row in g]
 return [row[:len(g[0])] 
  for row in [r * (len(g[0])//{period} + 1) for r in pattern]][:len(g)]
"""
        return None

    def handle_horizontal_stripes(self, task_data, analysis):
        """Create horizontal stripes"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find repeating row pattern
        for period in range(1, min(5, out.shape[0]+1)):
            pattern = out[:period,:]
            test = np.tile(pattern, (out.shape[0]//period + 1, 1))
            test = test[:out.shape[0], :]
            if np.array_equal(out, test):
                return f"""def p(g):
 pattern = g[:{period}]
 return (pattern * (len(g)//{period} + 1))[:len(g)]
"""
        return None

    def handle_majority_color_fill(self, task_data, analysis):
        """Fill with the majority color from input"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        colors, counts = np.unique(inp, return_counts=True)
        if len(colors) == 0: return None
        majority = colors[np.argmax(counts)]
        
        if np.all(out == majority):
            return f"""def p(g):
 from collections import Counter
 counts = Counter(x for row in g for x in row)
 majority = counts.most_common(1)[0][0]
 return [[majority]*len(g[0]) for _ in range(len(g))]
"""
        return None

    def handle_cross_lines(self, task_data, analysis):
        """Draw cross lines through center"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        center_row = inp.shape[0] // 2
        center_col = inp.shape[1] // 2
        test = np.zeros_like(inp)
        test[center_row,:] = out[center_row,0]
        test[:,center_col] = out[0,center_col]
        
        if np.array_equal(out, test):
            return """def p(g):
 h, w = len(g), len(g[0])
 res = [[0]*w for _ in range(h)]
 cr, cc = h//2, w//2
 color_v = g[0][cc]
 color_h = g[cr][0]
 res[cr] = [color_h]*w
 for row in res:
  row[cc] = color_v
 return res
"""
        return None

    def handle_color_swap(self, task_data, analysis):
        """Swap two specific colors"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        diff = out - inp
        changed = np.where(diff != 0)
        if len(changed[0]) == 0: return None
        
        # Find color pairs that were swapped
        color_pairs = set()
        for i,j in zip(*changed):
            src = inp[i,j]
            dst = out[i,j]
            color_pairs.add((src, dst))
            
        if len(color_pairs) != 2: return None
        (a1, a2), (b1, b2) = color_pairs
        if (a1 != b2) or (a2 != b1): return None
        
        return f"""def p(g):
 return [[{a2} if x=={a1} else {a1} if x=={a2} else x for x in row] for row in g]
"""

    def handle_crop_center(self, task_data, analysis):
        """Crop to center portion of grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Find crop boundaries
        for h in range(1, inp.shape[0]+1):
            for w in range(1, inp.shape[1]+1):
                if h > out.shape[0] or w > out.shape[1]: continue
                y0 = (inp.shape[0] - h) // 2
                x0 = (inp.shape[1] - w) // 2
                crop = inp[y0:y0+h, x0:x0+w]
                if crop.shape == out.shape and np.array_equal(crop, out):
                    return f"""def p(g):
 h, w = {h}, {w}
 y0 = (len(g) - h) // 2
 x0 = (len(g[0]) - w) // 2
 return [r[x0:x0+w] for r in g[y0:y0+h]]
"""
        return None

    def handle_single_color_output(self, task_data, analysis):
        """Output is a single color"""
        out = np.array(task_data['train'][0]['output'])
        if len(np.unique(out)) != 1: return None
        
        color = out[0,0]
        return f"""def p(g):
 return [[{color}]*len(g[0]) for _ in range(len(g))]
"""

    def handle_bounding_crop(self, task_data, analysis):
        """Crop to bounding box of non-zero elements"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        non_zero = np.where(inp > 0)
        if len(non_zero[0]) == 0: return None
        y0, y1 = min(non_zero[0]), max(non_zero[0])
        x0, x1 = min(non_zero[1]), max(non_zero[1])
        crop = inp[y0:y1+1, x0:x1+1]
        
        if np.array_equal(crop, out):
            return """def p(g):
 import numpy as np
 arr = np.array(g)
 non_zero = np.where(arr > 0)
 if len(non_zero[0]) == 0: return g
 y0, y1 = min(non_zero[0]), max(non_zero[0])
 x0, x1 = min(non_zero[1]), max(non_zero[1])
 return arr[y0:y1+1, x0:x1+1].tolist()
"""
        return None

    def handle_overlay_fill(self, task_data, analysis):
        """Fill areas surrounded by same color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Simple flood fill implementation
        filled = np.copy(inp)
        for i in range(1, inp.shape[0]-1):
            for j in range(1, inp.shape[1]-1):
                if inp[i,j] == 0:
                    neighbors = [inp[i-1,j], inp[i+1,j], inp[i,j-1], inp[i,j+1]]
                    if len(set(neighbors)) == 1 and neighbors[0] > 0:
                        filled[i,j] = neighbors[0]
                        
        if np.array_equal(out, filled):
            return """def p(g):
 h, w = len(g), len(g[0])
 res = [r[:] for r in g]
 for i in range(1, h-1):
  for j in range(1, w-1):
   if g[i][j] == 0:
    n = [g[i-1][j], g[i+1][j], g[i][j-1], g[i][j+1]]
    if len(set(n)) == 1 and n[0] > 0:
     res[i][j] = n[0]
 return res
"""
        return None

    def handle_remove_color(self, task_data, analysis):
        """Remove all instances of a specific color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find which color was removed
        in_colors = set(np.unique(inp))
        out_colors = set(np.unique(out))
        removed = in_colors - out_colors
        if len(removed) != 1: return None
        color = removed.pop()
        
        return f"""def p(g):
 return [[0 if x=={color} else x for x in row] for row in g]
"""

    def handle_color_mapping(self, task_data, analysis):
        """Map specific input colors to output colors"""
        color_map = {}
        for pos, changes in analysis['color_changes'].items():
            if len(changes) == 1:
                src, dest = next(iter(changes))
                if src not in color_map:
                    color_map[src] = dest
                elif color_map[src] != dest:
                    return None
        
        if not color_map:
            return None
            
        cases = "\n".join([f"    if x=={int(src)}: return {int(dest)}" 
                          for src, dest in color_map.items()])
        return f"""def p(g):
 return[[(lambda x:
{cases}
    else x)(x)for x in r]for r in g]
"""

    def handle_color_shift_plus_one(self, task_data, analysis):
        """Shift all colors up by 1 (with wrap around)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        shifted = np.where(inp > 0, (inp % 9) + 1, 0)
        if np.array_equal(out, shifted):
            return """def p(g):
 return [[(x % 9) + 1 if x > 0 else 0 for x in row] for row in g]
"""
        return None

    def handle_keep_colors(self, task_data, analysis):
        """Keep only certain colors, set others to 0"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        keep_colors = set(np.unique(out)) - {0}
        if len(keep_colors) == 0: return None
        
        test = np.where(np.isin(inp, list(keep_colors)), inp, 0)
        if np.array_equal(out, test):
            colors_str = ",".join(str(c) for c in keep_colors)
            return f"""def p(g):
 keep = {{{colors_str}}}
 return [[x if x in keep else 0 for x in row] for row in g]
"""
        return None

    def handle_grid_operations(self, task_data, analysis):
        """Handle grid operations like tiling, repeating, etc."""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if output is tiled version of input
        if out.shape[0] % inp.shape[0] == 0 and out.shape[1] % inp.shape[1] == 0:
            reps = (out.shape[0] // inp.shape[0], out.shape[1] // inp.shape[1])
            if np.array_equal(np.tile(inp, reps), out):
                return f"""def p(g):
 import numpy as np
 return np.tile(g, ({reps[0]}, {reps[1]})).tolist()
"""
        return None

    def handle_object_operations(self, task_data, analysis):
        """Handle operations on objects (move, scale, etc.)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check if objects were moved by fixed amount
        in_objs = inp > 0
        out_objs = out > 0
        
        # Find translation that aligns objects
        for dy in range(-inp.shape[0], inp.shape[0]):
            for dx in range(-inp.shape[1], inp.shape[1]):
                shifted = np.roll(in_objs, (dy, dx), axis=(0,1))
                if np.array_equal(shifted, out_objs):
                    # Now check if colors were preserved
                    color_match = True
                    for i in range(inp.shape[0]):
                        for j in range(inp.shape[1]):
                            if shifted[i,j]:
                                src_i, src_j = (i - dy) % inp.shape[0], (j - dx) % inp.shape[1]
                                if inp[src_i, src_j] != out[i,j]:
                                    color_match = False
                                    break
                        if not color_match: break
                    
                    if color_match:
                        return f"""def p(g):
 h, w = len(g), len(g[0])
 res = [[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   src_i, src_j = (i - {dy}) % h, (j - {dx}) % w
   res[i][j] = g[src_i][src_j]
 return res
"""
        return None

    def handle_pattern_replication(self, task_data, analysis):
        """Replicate a pattern in specific positions"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Try to find a small pattern that's repeated
        for h in range(1, min(4, inp.shape[0]+1)):
            for w in range(1, min(4, inp.shape[1]+1)):
                pattern = inp[:h,:w]
                # Check if output consists of this pattern repeated
                if (out.shape[0] % h == 0) and (out.shape[1] % w == 0):
                    reps = (out.shape[0] // h, out.shape[1] // w)
                    if np.array_equal(np.tile(pattern, reps), out):
                        return f"""def p(g):
 pattern = [r[:{w}] for r in g[:{h}]]
 return [row for _ in range({reps[0]}) 
         for row in [r*{reps[1]} for r in pattern]]
"""
        return None

    def handle_mirror_symmetry(self, task_data, analysis):
        """Handle mirror symmetry operations"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(out, np.fliplr(inp)):
            return """def p(g):
 return [row[::-1] for row in g]
"""
        if np.array_equal(out, np.flipud(inp)):
            return """def p(g):
 return g[::-1]
"""
        return None

    def handle_arithmetic_operations(self, task_data, analysis):
        """Handle arithmetic operations on colors"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check for addition/subtraction of constant
        diff = out - inp
        if np.all(diff == diff[0,0]):
            return f"""def p(g):
 return [[x + {diff[0,0]} for x in row] for row in g]
"""
        
        # Check for multiplication by constant
        with np.errstate(divide='ignore', invalid='ignore'):
            ratio = out / inp
            if np.all(np.nan_to_num(ratio) == np.nan_to_num(ratio[0,0])):
                return f"""def p(g):
 return [[x * {ratio[0,0]} for x in row] for row in g]
"""
        
        # Check for modulo operation
        if np.all(inp > 0):
            mod = out[0,0] - inp[0,0]
            if mod < 0 and np.all(out == inp + mod):
                return f"""def p(g):
 return [[(x + {mod}) for x in row] for row in g]
"""
        return None

    def handle_conditional_operations(self, task_data, analysis):
        """Handle conditional operations on cells"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check for thresholding
        threshold = None
        for t in range(1, 10):
            test = np.where(inp >= t, inp, 0)
            if np.array_equal(test, out):
                threshold = t
                break
        if threshold is not None:
            return f"""def p(g):
 return [[x if x >= {threshold} else 0 for x in row] for row in g]
"""
        
        # Check for conditional color change
        for cond_val in range(10):
            for result_val in range(10):
                test = np.where(inp == cond_val, result_val, inp)
                if np.array_equal(test, out):
                    return f"""def p(g):
 return [[{result_val} if x == {cond_val} else x for x in row] for row in g]
"""
        return None

    def handle_upscale_nearest(self, task_data, analysis):
        """Upscale using nearest neighbor interpolation"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check integer scaling factors
        for h_scale in range(1, 4):
            for w_scale in range(1, 4):
                if (inp.shape[0]*h_scale == out.shape[0] and 
                    inp.shape[1]*w_scale == out.shape[1]):
                    test = np.repeat(np.repeat(inp, h_scale, axis=0), w_scale, axis=1)
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return [[x for x in row for _ in range({w_scale})] 
         for row in g for _ in range({h_scale})]
"""
        return None

    def handle_downscale_sample(self, task_data, analysis):
        """Downscale by sampling pixels"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check integer scaling factors
        for h_scale in range(1, 4):
            for w_scale in range(1, 4):
                if (inp.shape[0]//h_scale == out.shape[0] and 
                    inp.shape[1]//w_scale == out.shape[1]):
                    test = inp[::h_scale, ::w_scale]
                    if np.array_equal(test, out):
                        return f"""def p(g):
 return [row[::{w_scale}] for row in g[::{h_scale}]]
"""
        return None

    def handle_fill_gaps_rowwise(self, task_data, analysis):
        """Fill gaps between objects row-wise"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check if gaps between non-zero pixels are filled with nearest color
        test = np.copy(inp)
        for i in range(inp.shape[0]):
            row = inp[i]
            non_zero = np.where(row > 0)[0]
            if len(non_zero) < 2: continue
            
            for j in range(1, len(non_zero)):
                left, right = non_zero[j-1], non_zero[j]
                if right - left > 1:
                    test[i, left:right] = row[left]
                    
        if np.array_equal(test, out):
            return """def p(g):
 for row in g:
  non_zero = [j for j,x in enumerate(row) if x > 0]
  for j in range(1, len(non_zero)):
   left, right = non_zero[j-1], non_zero[j]
   if right - left > 1:
    for k in range(left, right):
     row[k] = row[left]
 return g
"""
        return None

    def handle_background_to_majority(self, task_data, analysis):
        """Change background to majority color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find majority color in input
        colors, counts = np.unique(inp, return_counts=True)
        if len(colors) < 2: return None
        majority = colors[np.argmax(counts)]
        
        # Find background color (most common color touching edges)
        edge_pixels = np.concatenate([
            inp[0,:], inp[-1,:], inp[:,0], inp[:,-1]])
        bg_color = np.bincount(edge_pixels).argmax()
        
        # Check if output is input with bg_color replaced by majority
        test = np.where(inp == bg_color, majority, inp)
        if np.array_equal(test, out):
            return f"""def p(g):
 majority = {majority}
 bg_color = {bg_color}
 return [[majority if x == bg_color else x for x in row] for row in g]
"""
        return None

    def handle_keep_dominant_color(self, task_data, analysis):
        """Keep only the dominant color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Find dominant color in input
        colors, counts = np.unique(inp, return_counts=True)
        if len(colors) < 2: return None
        dominant = colors[np.argmax(counts)]
        
        # Check if output is binary mask of dominant color
        test = np.where(inp == dominant, dominant, 0)
        if np.array_equal(test, out):
            return f"""def p(g):
 dominant = {dominant}
 return [[dominant if x == dominant else 0 for x in row] for row in g]
"""
        return None

    def handle_extract_first_nonempty_rowcol(self, task_data, analysis):
        """Extract first non-empty row or column"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check for first non-empty row
        for i in range(inp.shape[0]):
            if np.any(inp[i] > 0):
                if out.shape == (1, inp.shape[1]) and np.array_equal([inp[i]], out):
                    return f"""def p(g):
 for i,row in enumerate(g):
  if any(x > 0 for x in row):
   return [row]
 return g
"""
                break
                
        # Check for first non-empty column
        for j in range(inp.shape[1]):
            if np.any(inp[:,j] > 0):
                if out.shape == (inp.shape[0], 1) and np.array_equal(inp[:,j:j+1], out):
                    return f"""def p(g):
 for j in range(len(g[0])):
  if any(g[i][j] > 0 for i in range(len(g))):
   return [[row[j]] for row in g]
 return g
"""
                break
        return None

    def handle_sort_rows_by_density(self, task_data, analysis):
        """Sort rows by density of non-zero pixels"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Calculate row densities
        densities = [np.count_nonzero(row) for row in inp]
        sorted_indices = np.argsort(densities)
        
        # Check if output is sorted version of input
        if np.array_equal(inp[sorted_indices], out):
            return """def p(g):
 densities = [sum(1 for x in row if x > 0) for row in g]
 sorted_indices = sorted(range(len(g)), key=lambda i: densities[i])
 return [g[i][:] for i in sorted_indices]
"""
        return None

    def handle_sort_cols_by_density(self, task_data, analysis):
        """Sort columns by density of non-zero pixels"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Calculate column densities
        densities = [np.count_nonzero(inp[:,j]) for j in range(inp.shape[1])]
        sorted_indices = np.argsort(densities)
        
        # Check if output is sorted version of input
        if np.array_equal(inp[:, sorted_indices], out):
            return """def p(g):
 densities = [sum(1 for row in g if row[j] > 0) for j in range(len(g[0]))]
 sorted_indices = sorted(range(len(g[0])), key=lambda j: densities[j])
 return [[row[j] for j in sorted_indices] for row in g]
"""
        return None

    def handle_complete_by_mirror_half(self, task_data, analysis):
        """Complete grid by mirroring half"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check horizontal mirror
        if inp.shape[0] * 2 == out.shape[0] and inp.shape[1] == out.shape[1]:
            mirrored = np.concatenate([inp, inp[::-1]])
            if np.array_equal(mirrored, out):
                return """def p(g):
 return g + g[::-1]
"""
        
        # Check vertical mirror
        if inp.shape[1] * 2 == out.shape[1] and inp.shape[0] == out.shape[0]:
            mirrored = np.concatenate([inp, inp[:,::-1]], axis=1)
            if np.array_equal(mirrored, out):
                return """def p(g):
 return [row + row[::-1] for row in g]
"""
        return None

    def handle_palette_column(self, task_data, analysis):
        """Apply palette from a specific column"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find a column that could serve as palette
        for col_idx in range(inp.shape[1]):
            palette_col = inp[:, col_idx]
            colors = {c:i for i,c in enumerate(palette_col) if c > 0}
            if not colors: continue
            
            # Try applying this palette
            test = np.zeros_like(inp)
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if inp[i,j] in colors:
                        test[i,j] = palette_col[colors[inp[i,j]]]
            
            if np.array_equal(out, test):
                return f"""def p(g):
 palette = [row[{col_idx}] for row in g]
 colors = {{c:i for i,c in enumerate(palette) if c>0}}
 return [[palette[colors.get(x,0)] for x in row] for row in g]
"""
        return None

    def handle_uniform_row_fill(self, task_data, analysis):
        """Fill each row with its most common color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            row = inp[i]
            counts = np.bincount(row)
            if len(counts) > 1:  # Ignore 0
                mode = np.argmax(counts[1:]) + 1
                test[i] = mode
                
        if np.array_equal(test, out):
            return """def p(g):
 for row in g:
  counts = [0]*10
  for x in row:
   counts[x] += 1
  mode = counts[1:].index(max(counts[1:])) + 1
  for j in range(len(row)):
   row[j] = mode
 return g
"""
        return None

    def handle_uniform_col_fill(self, task_data, analysis):
        """Fill each column with its most common color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        test = np.zeros_like(inp)
        for j in range(inp.shape[1]):
            col = inp[:,j]
            counts = np.bincount(col)
            if len(counts) > 1:  # Ignore 0
                mode = np.argmax(counts[1:]) + 1
                test[:,j] = mode
                
        if np.array_equal(test, out):
            return """def p(g):
 for j in range(len(g[0])):
  counts = [0]*10
  for i in range(len(g)):
   counts[g[i][j]] += 1
  mode = counts[1:].index(max(counts[1:])) + 1
  for i in range(len(g)):
   g[i][j] = mode
 return g
"""
        return None

    def handle_draw_frame(self, task_data, analysis):
        """Draw a frame around the grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check if output is input with frame added
        frame_color = out[0,0]
        test = np.copy(inp)
        test[0,:] = frame_color
        test[-1,:] = frame_color
        test[:,0] = frame_color
        test[:,-1] = frame_color
        
        if np.array_equal(test, out):
            return f"""def p(g):
 color = {frame_color}
 g[0] = [color]*len(g[0])
 g[-1] = [color]*len(g[0])
 for row in g:
  row[0] = color
  row[-1] = color
 return g
"""
        return None

    def handle_main_diag_line(self, task_data, analysis):
        """Draw line along main diagonal"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        test = np.copy(inp)
        color = out[0,0] if out[0,0] != inp[0,0] else out[-1,-1]
        for i in range(min(inp.shape[0], inp.shape[1])):
            test[i,i] = color
            
        if np.array_equal(test, out):
            return f"""def p(g):
 color = {color}
 for i in range(min(len(g), len(g[0]))):
  g[i][i] = color
 return g
"""
        return None

    def handle_anti_diag_line(self, task_data, analysis):
        """Draw line along anti-diagonal"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        test = np.copy(inp)
        color = out[0,-1] if out[0,-1] != inp[0,-1] else out[-1,0]
        for i in range(min(inp.shape[0], inp.shape[1])):
            test[i,inp.shape[1]-1-i] = color
            
        if np.array_equal(test, out):
            return f"""def p(g):
 color = {color}
 for i in range(min(len(g), len(g[0]))):
  g[i][len(g[0])-1-i] = color
 return g
"""
        return None

    def handle_border_to_zero(self, task_data, analysis):
        """Set border pixels to zero"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        test = np.copy(inp)
        test[0,:] = 0
        test[-1,:] = 0
        test[:,0] = 0
        test[:,-1] = 0
        
        if np.array_equal(test, out):
            return """def p(g):
 g[0] = [0]*len(g[0])
 g[-1] = [0]*len(g[0])
 for row in g:
  row[0] = 0
  row[-1] = 0
 return g
"""
        return None

    def handle_single_object_translate(self, task_data, analysis):
        """Translate a single object by fixed amount"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find translation that moves the object
        in_obj = inp > 0
        out_obj = out > 0
        
        # Find translation vector
        in_pos = np.argwhere(in_obj)
        out_pos = np.argwhere(out_obj)
        if len(in_pos) != len(out_pos): return None
        
        if len(in_pos) == 0: return None
        dy, dx = out_pos[0] - in_pos[0]
        
        # Verify all points moved by same amount
        for i in range(1, len(in_pos)):
            if not np.array_equal(out_pos[i] - in_pos[i], [dy, dx]):
                return None
                
        # Verify colors were preserved
        for i,j in out_pos:
            if out[i,j] != inp[i-dy, j-dx]:
                return None
                
        return f"""def p(g):
 h, w = len(g), len(g[0])
 res = [[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   ni, nj = i - {dy}, j - {dx}
   if 0 <= ni < h and 0 <= nj < w:
    res[i][j] = g[ni][nj]
 return res
"""

    def handle_object_count_row(self, task_data, analysis):
        """Count objects in each row and use as color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Count objects in each row (connected components)
        test = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            row = inp[i]
            objects = []
            current = None
            for j in range(len(row)):
                if row[j] > 0 and (j == 0 or row[j-1] == 0):
                    objects.append(row[j])
            test[i] = len(objects)
            
        if np.array_equal(test, out):
            return """def p(g):
 for i,row in enumerate(g):
  count = 0
  for j in range(len(row)):
   if row[j] > 0 and (j == 0 or row[j-1] == 0):
    count += 1
  for j in range(len(row)):
   row[j] = count
 return g
"""
        return None

    def handle_remove_small_objects(self, task_data, analysis):
        """Remove small objects below size threshold"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        labeled, n = label(inp > 0)
        if n == 0: return None
        
        # Find size threshold
        sizes = [np.sum(labeled == i) for i in range(1, n+1)]
        remaining_sizes = [np.sum((labeled == i+1) & (out > 0)) for i in range(n)]
        
        # Find minimum size of remaining objects
        min_size = min(sizes[i] for i in range(n) if remaining_sizes[i] > 0) if any(remaining_sizes) else 0
        
        test = np.zeros_like(inp)
        for i in range(1, n+1):
            if sizes[i-1] >= min_size:
                test[labeled == i] = inp[labeled == i]
                
        if np.array_equal(test, out):
            return f"""def p(g):
 from scipy.ndimage import label
 import numpy as np
 arr = np.array(g)
 labeled, n = label(arr > 0)
 if n == 0: return g
 sizes = [np.sum(labeled == i) for i in range(1, n+1)]
 min_size = min(sizes) if sizes else 0
 res = np.zeros_like(arr)
 for i in range(1, n+1):
  if sizes[i-1] >= min_size:
   res[labeled == i] = arr[labeled == i]
 return res.tolist()
"""
        return None
    
    def handle_rotate_90(self, task_data, analysis):
        """Rotate grid 90 degrees clockwise"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.rot90(inp, -1), out):
            return """def p(g):
 return [list(r) for r in zip(*g[::-1])]
"""
        return None

    def handle_rotate_180(self, task_data, analysis):
        """Rotate grid 180 degrees"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.rot90(inp, 2), out):
            return """def p(g):
 return [r[::-1] for r in g[::-1]]
"""
        return None

    def handle_rotate_270(self, task_data, analysis):
        """Rotate grid 270 degrees clockwise (90 counter-clockwise)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.rot90(inp, 1), out):
            return """def p(g):
 return [list(r) for r in zip(*g)][::-1]
"""
        return None

    def handle_flip_horizontal(self, task_data, analysis):
        """Flip grid horizontally"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.fliplr(inp), out):
            return """def p(g):
 return [r[::-1] for r in g]
"""
        return None

    def handle_flip_vertical(self, task_data, analysis):
        """Flip grid vertically"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        if np.array_equal(np.flipud(inp), out):
            return """def p(g):
 return g[::-1]
"""
        return None

    def handle_invert_colors(self, task_data, analysis):
        """Invert all non-zero colors (1 becomes 9, 2 becomes 8, etc.)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        inverted = np.where(inp > 0, 10 - inp, 0)
        if np.array_equal(out, inverted):
            return """def p(g):
 return[[10-x if x>0 else 0 for x in r] for r in g]
"""
        return None

    def handle_shift_colors(self, task_data, analysis):
        """Shift all colors by fixed amount (with wrap-around)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find shift amount
        diff = out - inp
        non_zero = inp > 0
        if np.all(diff[non_zero] == diff[non_zero][0]):
            shift = diff[non_zero][0] % 9
            return f"""def p(g):
 return [[(x + {shift}) % 10 if x > 0 else 0 for x in row] for row in g]
"""
        return None

    def handle_threshold_colors(self, task_data, analysis):
        """Threshold colors above/below certain value"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check for thresholding
        for thresh in range(1, 10):
            for mode in ['above', 'below']:
                if mode == 'above':
                    test = np.where(inp >= thresh, inp, 0)
                else:
                    test = np.where(inp <= thresh, inp, 0)
                
                if np.array_equal(test, out):
                    return f"""def p(g):
 return [[x if x {'>' if mode=='above' else '<'}= {thresh} else 0 for x in row] for row in g]
"""
        return None

    def handle_expand_borders(self, task_data, analysis):
        """Expand borders of objects by 1 pixel"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        expanded = np.copy(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i,j] > 0:
                    for di, dj in [(-1,0),(1,0),(0,-1),(0,1)]:
                        ni, nj = i+di, j+dj
                        if 0 <= ni < inp.shape[0] and 0 <= nj < inp.shape[1]:
                            expanded[ni,nj] = inp[i,j]
                            
        if np.array_equal(out, expanded):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:] for r in g]
 for i in range(h):
  for j in range(w):
   if g[i][j]>0:
    for di,dj in [(-1,0),(1,0),(0,-1),(0,1)]:
     ni,nj=i+di,j+dj
     if 0<=ni<h and 0<=nj<w: res[ni][nj]=g[i][j]
 return res
"""
        return None

    def handle_shrink_objects(self, task_data, analysis):
        """Shrink objects by removing border pixels"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        test = np.copy(inp)
        for i in range(1, inp.shape[0]-1):
            for j in range(1, inp.shape[1]-1):
                if inp[i,j] > 0 and any(inp[i+di,j+dj] == 0 
                   for di,dj in [(-1,0),(1,0),(0,-1),(0,1)]):
                    test[i,j] = 0
                    
        if np.array_equal(test, out):
            return """def p(g):
 h,w=len(g),len(g[0])
 res=[r[:] for r in g]
 for i in range(1,h-1):
  for j in range(1,w-1):
   if g[i][j]>0 and any(g[i+di][j+dj]==0
      for di,dj in [(-1,0),(1,0),(0,-1),(0,1)]):
    res[i][j]=0
 return res
"""
        return None

    def handle_connect_components(self, task_data, analysis):
        """Connect nearby components of the same color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        connected = np.copy(inp)
        for color in np.unique(inp):
            if color == 0: continue
            mask = (inp == color)
            labeled, n = label(mask)
            if n <= 1: continue
            
            for i in range(inp.shape[0]):
                for j in range(inp.shape[1]):
                    if labeled[i,j] > 0:
                        for di, dj in [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]:
                            ni, nj = i+di, j+dj
                            if 0 <= ni < inp.shape[0] and 0 <= nj < inp.shape[1]:
                                if inp[ni,nj] == color:
                                    connected[ni,nj] = color
                                    
        if np.array_equal(out, connected):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 arr = np.array(g)
 res = arr.copy()
 for color in np.unique(arr):
  if color == 0: continue
  mask = (arr == color)
  labeled, n = label(mask)
  if n <= 1: continue
  for i in range(arr.shape[0]):
   for j in range(arr.shape[1]):
    if labeled[i,j] > 0:
     for di,dj in [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]:
      ni,nj=i+di,j+dj
      if 0<=ni<arr.shape[0] and 0<=nj<arr.shape[1]:
       if arr[ni,nj] == color: res[ni,nj] = color
 return res.tolist()
"""
        return None

    def handle_extract_largest_object(self, task_data, analysis):
        """Keep only the largest connected object"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        labeled, n = label(inp > 0)
        if n == 0: return None
        
        sizes = [np.sum(labeled == i) for i in range(1, n+1)]
        largest = np.argmax(sizes) + 1
        result = np.where(labeled == largest, inp, 0)
        
        if np.array_equal(out, result):
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 arr = np.array(g)
 labeled, n = label(arr > 0)
 if n == 0: return g
 sizes = [np.sum(labeled == i) for i in range(1, n+1)]
 largest = np.argmax(sizes) + 1
 res = np.where(labeled == largest, arr, 0)
 return res.tolist()
"""
        return None

    def handle_remove_background(self, task_data, analysis):
        """Remove background (most common edge color)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find background color (most common on edges)
        edge_pixels = np.concatenate([
            inp[0,:], inp[-1,:], inp[:,0], inp[:,-1]])
        bg_color = np.bincount(edge_pixels).argmax()
        
        test = np.where(inp == bg_color, 0, inp)
        if np.array_equal(test, out):
            return f"""def p(g):
 bg_color = {bg_color}
 return [[0 if x == bg_color else x for x in row] for row in g]
"""
        return None

    def handle_keep_only_background(self, task_data, analysis):
        """Keep only background (most common edge color)"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Find background color (most common on edges)
        edge_pixels = np.concatenate([
            inp[0,:], inp[-1,:], inp[:,0], inp[:,-1]])
        bg_color = np.bincount(edge_pixels).argmax()
        
        test = np.where(inp == bg_color, inp, 0)
        if np.array_equal(test, out):
            return f"""def p(g):
 bg_color = {bg_color}
 return [[x if x == bg_color else 0 for x in row] for row in g]
"""
        return None

    def handle_color_gradient(self, task_data, analysis):
        """Apply color gradient based on position"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check horizontal gradient
        if np.all(out[:,0] <= out[:,-1]):
            colors = sorted(set(out.ravel()))
            if len(colors) > 1:
                return """def p(g):
 return [[1 + (x * 8) // len(g[0]) for x in range(len(g[0]))] for _ in g]
"""
        
        # Check vertical gradient
        if np.all(out[0,:] <= out[-1,:]):
            colors = sorted(set(out.ravel()))
            if len(colors) > 1:
                return """def p(g):
 return [[1 + (y * 8) // len(g)) for _ in g[0]] for y in range(len(g))]
"""
        return None

    def handle_histogram_equalization(self, task_data, analysis):
        """Equalize color histogram"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        in_colors, in_counts = np.unique(inp, return_counts=True)
        out_colors, out_counts = np.unique(out, return_counts=True)
        
        if len(in_colors) == len(out_colors) and np.all(in_counts == out_counts):
            color_map = {in_colors[i]:out_colors[i] for i in range(len(in_colors))}
            cases = "\n".join([f"    if x=={src}: return {dest}" for src,dest in color_map.items()])
            return f"""def p(g):
 return[[(lambda x:
{cases}
    else x)(x)for x in r]for r in g]
"""
        return None

    def handle_fill_enclosed_areas(self, task_data, analysis):
        """Fill completely enclosed areas using XOR instead of subtraction"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: 
            return None
        
        # Changed from subtraction to XOR
        filled = binary_dilation(inp > 0) ^ binary_erosion(inp > 0)
        if np.array_equal(out, np.where(filled, inp, 0)):
            return """def p(g):
        from scipy.ndimage import binary_dilation, binary_erosion
        import numpy as np
        arr = np.array(g)
        filled = binary_dilation(arr > 0) ^ binary_erosion(arr > 0)
        return np.where(filled, arr, 0).tolist()
    """
        return None

    def handle_remove_isolated_pixels(self, task_data, analysis):
        """Remove isolated pixels"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        isolated = inp & ~binary_dilation(binary_erosion(inp))
        if np.array_equal(out, np.where(isolated, 0, inp)):
            return """def p(g):
 from scipy.ndimage import binary_dilation, binary_erosion
 import numpy as np
 arr = np.array(g)
 isolated = arr & ~binary_dilation(binary_erosion(arr))
 return np.where(isolated, 0, arr).tolist()
"""
        return None

    def handle_find_contours(self, task_data, analysis):
        """Find contours of objects"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        contours = np.zeros_like(inp)
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                if inp[i,j] > 0:
                    neighbors = [(i+1,j),(i-1,j),(i,j+1),(i,j-1)]
                    if any(not (0<=y<inp.shape[0] and 0<=x<inp.shape[1]) or inp[y,x]==0 
                       for y,x in neighbors):
                        contours[i,j] = inp[i,j]
                        
        if np.array_equal(out, contours):
            return """def p(g):
 h,w=len(g),len(g[0])
 return[[g[i][j] if g[i][j] and any((y<0 or y>=h or x<0 or x>=w or g[y][x]==0)
        for y,x in ((i+1,j),(i-1,j),(i,j+1),(i,j-1))) 
        else 0 for j in range(w)] for i in range(h)]
"""
        return None

    def handle_scale_objects(self, task_data, analysis):
        """Scale objects by a factor"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        # Check if output is scaled version of input
        if out.shape[0] % inp.shape[0] == 0 and out.shape[1] % inp.shape[1] == 0:
            scale_h = out.shape[0] // inp.shape[0]
            scale_w = out.shape[1] // inp.shape[1]
            test = np.kron(inp, np.ones((scale_h, scale_w)))
            if np.array_equal(test, out):
                return f"""def p(g):
 import numpy as np
 return np.kron(g, np.ones(({scale_h}, {scale_w}))).tolist()
"""
        return None

    def handle_align_objects(self, task_data, analysis):
        """Align objects to grid"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        # Check if objects are aligned to some grid
        for grid_size in range(2, min(inp.shape)//2 + 1):
            test = np.zeros_like(inp)
            for i in range(0, inp.shape[0], grid_size):
                for j in range(0, inp.shape[1], grid_size):
                    if np.any(inp[i:i+grid_size,j:j+grid_size] > 0):
                        test[i:i+grid_size,j:j+grid_size] = inp[i:i+grid_size,j:j+grid_size]
            
            if np.array_equal(test, out):
                return f"""def p(g):
 grid_size = {grid_size}
 res = [[0]*len(g[0]) for _ in range(len(g))]
 for i in range(0, len(g), grid_size):
  for j in range(0, len(g[0]), grid_size):
   if any(g[y][x] > 0 for y in range(i, min(i+grid_size, len(g)))
                          for x in range(j, min(j+grid_size, len(g[0])))):
    for y in range(i, min(i+grid_size, len(g))):
     for x in range(j, min(j+grid_size, len(g[0]))):
      res[y][x] = g[y][x]
 return res
"""
        return None

    def handle_count_objects(self, task_data, analysis):
        """Count objects and use count as output"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        
        labeled, n = label(inp > 0)
        if n == 0: return None
        
        if out.shape == (1,1) and out[0,0] == n:
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 labeled, n = label(np.array(g) > 0)
 return [[n]]
"""
        return None

    def handle_separate_objects(self, task_data, analysis):
        """Separate objects by color"""
        inp, out = (np.array(task_data['train'][0][k]) for k in ('input','output'))
        if inp.shape != out.shape: return None
        
        labeled, n = label(inp > 0)
        if n < 2: return None
        
        # Check if each object is assigned a unique color
        colors = set()
        for i in range(1, n+1):
            obj_colors = set(out[labeled == i])
            if len(obj_colors) != 1:
                return None
            colors.add(obj_colors.pop())
        
        if len(colors) == n:
            return """def p(g):
 from scipy.ndimage import label
 import numpy as np
 arr = np.array(g)
 labeled, n = label(arr > 0)
 res = np.zeros_like(arr)
 for i in range(1, n+1):
  res[labeled == i] = i
 return res.tolist()
"""
        return None
    def generate_solution(self, task_data):
        analysis = self.analyze_task(task_data)
        for handler in self.pattern_handlers:
            solution = handler(task_data, analysis)
            if solution and self.verify_solution(solution, task_data):
                return solution
        
        return """def p(g):
 return [row[:] for row in g]
"""
    
    def verify_solution(self, solution_code, task_data):
        try:
            exec(solution_code, globals())
            for example in task_data['train'][:2]:  # Check first few training examples
                input_grid = example['input']
                expected = example['output']
                actual = p(input_grid)
                if actual != expected:
                    return False
            return True
        except:
            return False



In [ ]:
import os
import json
import zipfile
from tqdm import tqdm
from rich import print
import numpy as np
from collections import defaultdict
from scipy.ndimage import label, binary_dilation, binary_erosion

# Predefined solutions for specific tasks
PREDEFINED_SOLUTIONS = {
    1: """def p(m):
 s=len(m)
 return[[m[i%s][j%s]if m[i//s][j//s]else 0for j in range(s*s)]for i in range(s*s)]""",
    2: """from collections import*
def p(a):
 if not a or not a[0]:return[]
 r,c=len(a),len(a[0])
 q=deque([(i,j)for i in range(r)for j in range(c)if(i*j==0 or i==r-1 or j==c-1)and a[i][j]==0])
 while q:
  x,y=q.popleft();a[x][y]=1
  for dx,dy in[(0,1),(0,-1),(1,0),(-1,0)]:
   nx,ny=x+dx,y+dy
   if 0<=nx<r and 0<=ny<c and a[nx][ny]==0:q.append((nx,ny));a[nx][ny]=1
 return[[4 if v==0 else 0 if v==1 else v for v in row]for row in a]""",
    3: """def p(g):
  return [[c*2 for c in r] for r in g + (g[:3] if g[1] == g[4] else g[2:5])]""",
    4: """def p(g):
  s={}
  E=enumerate
  for r,R in E(g):
    for c,C in E(R):
      if C:s.setdefault(C,[]).append((r,c))
  o=[[0]*len(g[0])for _ in g]
  for C,P in s.items():
    mr,mc=max(r for r,_ in P),max(c for _,c in P)
    for r,c in P:o[r][c if r==mr or c==mc else c+1]=C
  return o""",
    6: """def p(g):
 S=[[0,0,0],[0,0,0],[0,0,0]]
 for r in range(3):
  for c in range(3):
   if g[r][c]==g[r][c+4] and g[r][c]>0:
    S[r][c]=2
 return S""",
    7: """def p(g):
 E=enumerate
 m=len({v for row in g for v in row if v})
 s=[0]*m
 for i,row in E(g):
  for j,v in E(row):
   if v:s[(i+j)%m]=v
 return[[s[(i+j)%m]for j in range(len(g[0]))]for i in range(len(g))]""",
    10: """def p(g):
 X=[]
 E=enumerate
 for r,R in E(g):
  for c,C in E(R):
   if C==5:
    if c not in X:X.append(c)
    g[r][c]=X.index(c)+1
 return g""",
    12: """def p(g):
 z=0
 E=enumerate
 X=[[C for i,C in E(R)] for R in g]
 for r,R in E(g):
  for c,C in E(R):
   try:
    if C!=0 and g[r][c+1]!=0 and g[r][c-1]!=0:
     for i in range(-2,3):
      for j in range(-2,3):
        if abs(i)==abs(j):X[r+i][c+j]=C
        elif 0 in [i,j]:X[r+i][c+j]=g[r][c-1]
   except:pass
 return X""",
    14: """from collections import Counter
def p(g):
 b=[x for R in g for x in R]
 C=Counter(b).most_common(3)
 C=[c for c in C if c[0]>0][-1][0]
 g=[R for R in g if C in R]
 T=[]
 for R in g:
  for i in range(len(R)):
   if R[i]==C:T.append(i)
 g=[R[min(T):max(T)+1] for R in g]
 return g""",
    15: """def p(g):
 z=0
 E=enumerate
 X=[[C for i,C in E(R)] for R in g]
 for r,R in E(g):
  for c,C in E(R):
   if C==2:
    for i in range(-1,2):
     for j in range(-1,2):
      try:
       if abs(i)==abs(j) and X[r+i][c+j]==0:X[r+i][c+j]=4
      except:pass
   if C==1:
    for i in range(-1,2):
     for j in range(-1,2):
      try:
       if 0 in [i,j] and X[r+i][c+j]==0:X[r+i][c+j]=7
      except:pass
 return X""",
    16: """def p(g):
 d={0:7,1:5,2:6,3:4,4:3,5:1,6:2,7:0,8:9,9:8}
 E=enumerate
 for r,R in E(g):
  for c,C in E(R):
   g[r][c]=d[C]
 return g""",
    24: """def p(g):
 X=[]
 Z=[]
 E=enumerate
 for r,R in E(g):
  if 1 in R:X.append([1 for _ in R])
  elif 3 in R:X.append([3 for _ in R])
  else:X.append(R[:])
  for c,C in E(R):
   if C==2:Z.append(c)
 for r,R in E(X):
  for c,C in E(R):
   if c in Z and C==0:
       X[r][c]=2
 return X""",
    26: """def p(g):
 S=[[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
 for r in range(5):
  for c in range(3):
   if g[r][c]==0 and g[r][c+4]==0:
    S[r][c]=8
 return S""",
    28: """def p(g):
 S=[[1,1,1,1,1,1,1,1,1,1],[1,0,0,0,0,0,0,0,0,1],[1,1,1,1,1,1,1,1,1,1],[1,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,1],[2,0,0,0,0,0,0,0,0,2],[2,0,0,0,0,0,0,0,0,2],[2,2,2,2,2,2,2,2,2,2],[2,0,0,0,0,0,0,0,0,2],[2,2,2,2,2,2,2,2,2,2]]
 S=[[i+10 if i>0 else 0 for i in R] for R in S]
 m1=max([max(R) for R in g[:5]])
 m2=max([max(R) for R in g[-5:]])
 S=[[m1 if i==11 else i for i in R] for R in S]
 S=[[m2 if i==12 else i for i in R] for R in S]
 return S""",
    31: """def p(g):
 E=enumerate
 c=[(i,j)for i,r in E(g)for j,x in E(r)if x]
 x0=min(j for _,j in c); x1=max(j for _,j in c)+1
 y0=min(i for i,_ in c); y1=max(i for i,_ in c)+1
 return[g[i][x0:x1] for i in range(y0,y1)]""",
    32: """def p(g):
 h,w=len(g),len(g[0])
 for c in range(w):
  u,i=[],0
  for r in range(h):u.append(g[r-h][c])
  u=[0 for x in u if x==0]+[x for x in u if x>0]
  for r in range(h):g[r-h][c]=u[r]
 return g""",
    41: """def p(g):
 z=0
 S=False
 E=enumerate
 for r,R in E(g):
  for c,C in E(R):
   if C!=0:
    if S==False:
     S=True
     z=int(C)
    else:
     S=False
     z=0
   else:g[r][c]=z
 return g""",
    43: """def p(g):
 E=enumerate
 h=len(g)-1
 w=len(g[0])-1
 for r,R in E(g):
  for c,C in E(R):
   if r>0 and c<h:
    if g[r][w]==5 and g[0][c]==5: g[r][c]=2
 return g""",
    45: """def p(g):
 res=[]
 for r in g:
  r=r[:]
  s={x for x in r if x!=0}
  for c in s:
   idx=[j for j,x in enumerate(r) if x==c]
   l,rgt=min(idx),max(idx)
   for j in range(l,rgt+1):
    if r[j]==0:r[j]=c
  res.append(r)
 return res""",
    47: """def p(g):
 X=[[0 for i in range(len(R))] for R in g]
 E=enumerate
 P=[]
 for r,R in E(g):
  for c,C in E(R):
   if C>0:
    for i in range(-9,10):
     for j in range(-9,10):
      try:
       if 0 in [i,j]:
        if [r+i,c+j] in P:X[r+i][c+j]=2
        else: X[r+i][c+j]=int(C)
        P.append([r+i,c+j])
      except:pass
 return X""",
    48: """def p(g):
 h,w =1,1
 y0=(len(g) - h)//2
 x0=(len(g[0])-w)//2
 return [r[x0:x0+w] for r in g[y0:y0+h]]""",
    52: """def p(g):
 X=[]
 for i in range(3):
  S = list(set(g[i]))
  if len(S)>1:X.append([0]*3)
  else:X.append([5]*3)
 return X""",
    53: """def p(g):
 h,w=len(g),len(g[0])
 res=[[0]*w for _ in range(h)]
 for i in range(h):
  for j in range(w):
   _i, _j = (i - -2) % h, (j - -3) % w
   res[i][j] = g[_i][_j]
 return res""",
    56: """def p(g):
 d={1:[[1,1,0],[1,0,1],[0,1,0]],2:[[1,0,1],[0,1,0],[1,0,1]],3:[[0,1,1],[0,1,1],[1,0,0]],6:[[0,1,0],[1,1,1],[0,1,0]]}
 g=[[1 if x>0 else 0 for x in R] for R in g]
 for k in d:
  if g==d[k]:return [[k]]""",
    57: """def p(g):
 X=[]
 T=[]
 for R in g:
  if max(R)>0:X.append(R)
 for R in X:
  for i in range(len(R)):
     if R[i]>0:T.append(i)
 X=[R[min(T):max(T)+1] for R in X]
 X=[R*2 for R in X]
 return X""",
    59: """def p(g):
 o=0
 X=[[0 if (i+1)%4>0 and (j+1)%4>0 else 5 for i in range(11)] for j in range(11)]
 d={'00':0,'01':0,'02':0,'10':0,'11':0,'12':0,'20':0,'21':0,'22':0}
 E=enumerate
 for r,R in E(g):
  for c,C in E(R):
   if C>0 and C!=5:
    o=int(C)
    d[str(r//4)+str(c//4)]+=1
 m=max([d[k] for k in d])
 for r,R in E(X):
  for c,C in E(R):
   if C==0 and d[str(r//4)+str(c//4)]==m:
    X[r][c]=o
 return X""",
    60: """def p(g):
 w=len(g[0])
 h=int((w-1)/2)
 E=enumerate
 for r,R in E(g):
  if max(R)>0:
   for x in range(h):
    g[r][x]= g[r][0]
    g[r][w-x-1]= g[r][w-1]
   g[r][h]=5
 return g""",
    67: """def p(g):
 return [R[:int(len(g[0])/3)] for R in g]""",
    72: """def p(g):
 for i,r in enumerate(g):
  if all(c==4 for c in r):
   t,b=g[:i],g[i+1:i+1+i]
   return[[3 if(t[r][c]!=0)!=(b[r][c]!=0)else 0 for c in range(len(t[0]))]for r in range(len(t))]""",
    73: """def p(g):
 X=[[5 if C==5 else 0 for C in R] for R in g]
 for r in range(len(g)):
  for c in range(len(g[r])):
   C=g[r][c]
   if C>0 and C!=5:
    X[len(g)-1][c]=C
 return X""",
    78: """def p(g):
 d={i:0 for i in range(len(g[0]))}
 X=[[1 if C==1 else 0 for C in R] for R in g]
 E=enumerate
 for r,R in E(g):
  for c,C in E(R):
   if C==2:d[c]+=1
 for r,R in E(X):
  for c,C in E(R):
   if C==0 and d[c]>0:
    X[r][c]=2
    d[c]-=1
 return X""",
    81: """from collections import *
def p(g):
 X=[[8 if C==8 else 0 for C in R] for R in g]
 for r in range(len(g)-1):
  for c in range(len(g[0])-1):
   a=[g[r][c:c+2],g[r+1][c:c+2]]
   b=[x for R in a for x in R]
   C=Counter(b).most_common(1)
   if C[0][1]==3 and C[0][0]!=0:
    for i in range(r,r+2):
     for j in range(c,c+2):
      if X[i][j]==0:X[i][j]=1
 return X""",
    83: """def p(g):
 t=[r+r[::-1]for r in g]
 b=[r+r[::-1]for r in g[::-1]]
 return t+b""",
    84: """def p(g):
 w,h=len(g[0]),len(g)
 for c in range(1,w):
  g[h-1][c]=4
  g[h-c-1][c]=2
 return g""",
    87: """def p(g):return[r[::-1]for r in g[::-1]]""",
    95: """def p(g):
 E=enumerate
 for r,R in E(g):
  for c,C in E(R):
   if C==5:
    for i in range(r-1,r+2):
     for j in range(c-1,c+2):
      if [i,j]!=[r,c]:
       g[i][j]=1
 return g""",
    98: """def p(g):h,w=len(g),len(g[0]);return[[g[i][j] if g[i][j]!=0 and any(i+di<0 or i+di>=h or j+dj<0 or j+dj>=w or g[i+di][j+dj]==0 for di,dj in [(0,1),(0,-1),(1,0),(-1,0)]) else 0 for j in range(w)] for i in range(h)]""",
    100: """from collections import *
def p(g):
 X=[x for R in g for x in R]
 C=Counter(X).most_common(2)
 if C[0][0]==0:c=C[1][0]
 else:c=C[0][0]
 return [[c]*2 for i in range(2)]""",
    103: """def p(g):
 b=[[[1,0,1],[0,1,0],[1,0,1]],[[1,0,1],[1,0,1],[1,0,1]],
    [[0,0,0],[1,0,1],[0,0,0]],[[1,0,1],[1,1,1],[1,0,1]],
   [[0,0,0],[1,1,1],[0,0,0]],[[0,1,0],[1,1,1],[0,1,0]],
   [[0,1,0],[0,1,0],[0,1,0]],[[1,1,1],[1,1,1],[1,1,1]],
   [[0,1,0],[1,0,1],[0,1,0]],[[0,0,0],[0,1,0],[0,0,0]],
    [[0,1,0],[0,0,0],[0,1,0]],[[1,1,1],[0,1,0],[1,1,1]],
   [[1,1,1],[0,0,0],[0,0,0]],[[1,1,1],[0,0,0],[1,1,1]],
   [[1,1,1],[1,0,1],[1,1,1]],[[1,0,1],[0,1,0],[1,0,1]],
   [[1,0,1],[0,0,0],[1,0,1]]]
 g=[[1 if x>0 else 0 for x in R] for R in g]
 if g in b: return [[1]]
 else: return [[7]]""",
    113: """def p(g):return g[:len(g)//2]+g[:len(g)//2][::-1]""",
    116: """def p(g):return g[::-1]+g""",
    129: """from collections import *
def p(g):
 f=[x for r in g for x in r]
 m=Counter(f).most_common(1)[0][0]
 return[[m]*len(g[0]) for _ in g]""",
    132: """def p(g):
 E=enumerate
 H, W = len(g),len(g[0])
 res = [[0]*W for _ in range(H)]
 colors = {v for row in g for v in row if v}
 for c in colors:
  ys = [i for i,row in E(g) for v in row if v==c]
  xs = [j for i,row in E(g) for j,v in E(row) if v==c]
  y0,y1 = min(ys), max(ys)+1
  x0,x1 = min(xs), max(xs)+1
  for i in range(y0,y1):
   for j in range(x0,x1):
    res[i][j] = c
 return res""",
    135: """def p(g):return[g[i][6:9]for i in range(0,3)]""",
    140: """def p(g):return[r[::-1]for r in g[::-1]]""",
    142: """def p(g):
 t=[r+r[::-1]for r in g]
 b=[r+r[::-1]for r in g[::-1]]
 return t+b""",
    150: """def p(g):return[r[::-1]for r in g]""",
    152: """def p(input):
 n=len(input)
 o=[[0]*(2*n) for _ in range(2*n)]
 for i in range(n):
  for j in range(n):
   o[i][j]=input[i][j]
   o[i][2*n-j-1]=input[i][j]
   o[2*n-i-1][j]=input[i][j]
   o[2*n-i-1][2*n-j-1]=input[i][j]
 return o""",
    155: """def p(g):return g[::-1]""",
    164: """def p(g):return [row + row[::-1] for row in g]""",
    171: """def p(grid):
 rows = len(grid)
 cols = len(grid[0])
 output = [[0 for _ in range(cols)] for _ in range(rows)]
 for r in range(rows):
  for c in range(cols):
   if r==0 or r==rows-1 or c==0 or c==cols-1:
    output[r][c]=8
   else:
    output[r][c]=0      
 return output""",
    172: """def p(g):return g + g[::-1]""",
    179: """def p(g):return[list(r) for r in zip(*g)]""",
    186: """def p(g):
 d={1:[[2,0,0],[0,0,0],[0,0,0]],2:[[2,2,0],[0,0,0],[0,0,0]],3:[[2,2,2],[0,0,0],[0,0,0]],4:[[2,2,2],[0,2,0],[0,0,0]]}
 f=sum([i for s in g for i in s])
 return d[f]""",
    200: """def p(m):
 s,n=next((j,v)for j,v in enumerate(m[-1])if v)
 o=[[0]*10 for _ in range(10)]
 for i in range(10):
  for j in range(s,10):
   d=j-s
   if d%2==0:o[i][j]=n
   elif(i==0 and d%4==1)or(i==9 and d%4==3):o[i][j]=5
 return o""",
    207: """def p(g):
 d={}
 X=[[[g[0][0],g[0][1]],[g[1][0],g[1][1]]],
 [[g[3][0],g[3][1]],[g[4][0],g[4][1]]],
 [[g[0][3],g[0][4]],[g[1][3],g[1][4]]],
 [[g[3][3],g[3][4]],[g[4][3],g[4][4]]]]
 for k in X:
  k=str(k)
  if k in d:d[k]+=1
  else:d[k]=1
 for k in d:
  if d[k]==1:
   return eval(k)""",
    210: """def p(g):return g + g[::-1]""",
    211: """def p(g):
 g = [R[::-1] + R for R in g]
 fg = [g[2],g[1],g[0]]
 g = fg+g+fg
 return g""",
    220: """def p(g):
 d={8:4,2:1,3:6}
 E=enumerate
 X=[[C for i,C in E(R)] for R in g]
 for r,R in E(g):
  for c,C in E(R):
   if C>0:
    for i in range(-1,2):
     for j in range(-1,2):
      try:
       if [i,j]!=[0,0]:X[r+i][c+j]=d[C]
      except:pass
 return X""",
    223: """def p(g):
 import numpy as np
 return np.kron(g, np.ones((3, 3))).tolist()""",
    227: """def p(g):
 for i in range(4):
  for j in range(4):
   g[i][j]+=g[i+4][j]
 g=[[2 if C==0 else 1 for C in R] for R in g]
 g=[[0 if C!=2 else 2 for C in R] for R in g]
 g=g[:4]
 return g""",
    229: """from collections import *
def p(g):
 X=[x for R in g for x in R]
 C=Counter(X).most_common(1)
 c=C[0][0]
 g=[[C if C==c else 5 for C in R] for R in g]
 return g""",
    231: """def p(g):return[[g[i%5][j%6]for j in range(len(g[0])*2)]for i in range(len(g)*1)]""",
    232: """def p(g):
 E=enumerate
 for r,R in E(g):
  i,X,S=0,[],False
  for c,C in E(R):
   if C>0:
    X=[C,5]*20
    S=True
   if S:
    g[r][c]=X[i]
    i+=1
 return g""",
    236: """def p(g):
 for r in range(4):
  for c in range(4):
   g[r][c]+=g[r+5][c]
   if g[r][c]==3:g[r][c]=0
   elif g[r][c]>0:g[r][c]=3
 return g[:4]""",
    239: """from collections import *
def p(g):
 C=Counter([x for r in g for x in r]).most_common(9)
 h,w=C[0][1],len(C)
 g=[[0 for _ in range(w)] for _ in range(h)]
 for i in range(w):
  for j in range(C[i][1]):
   g[j][i]=C[i][0]
 return g""",
    241: """def p(g):return[list(r) for r in zip(*g)]""",
    249: """def p(input):
 return [row * 2 for row in input]""",
    254: """def p(g):
 h,w=len(g),len(g[0])
 s=[0 for _ in range(w)]
 for i in range(w):
  for j in range(h):
   if g[j][i]>0:s[i]+=1
   g[j][i]=0
 m=min([c for c in s if c>0])
 i=s.index(m)
 for j in range(s[i]):g[-(j+1)][i]=2
 i=s.index(max(s))
 for j in range(s[i]):g[-(j+1)][i]=1
 return g""",
    261: """def p(g):
 g=[g[-1]]+g[:len(g)-1]
 g=[[2 if C==8 else C for C in R] for R in g]
 return g""",
    276: """def p(g):
 m={6:2,7:7}
 return[[m.get(x,x)for x in r]for r in g]""",
    289: """import numpy as np
def p(g):
 X=[x for R in g for x in R]
 i=len(set(X))-1
 return np.kron(g,np.ones((i, i))).astype(int).tolist()""",
    297: """def p(g):
 h,w=len(g),len(g[0])
 C=g[0]*20
 for i in range(2,h):
  g[i]=[C[i-2] for _ in range(w)]
 return g""",
    300: """def p(m):
 E=enumerate
 a=[(i,j)for i,r in E(m)for j,v in E(r)if v]
 if not a:return[]
 from collections import Counter as C
 v=C(m[i][j]for i,j in a).most_common(1)[0][0]
 x=[(i,j)for i,j in a if m[i][j]==v]
 r0,c0=min(i for i,_ in x),min(j for _,j in x)
 r1,c1=max(i for i,_ in x)+1,max(j for _,j in x)+1
 return[m[i][c0:c1]for i in range(r0,r1)]""",
    301: """from collections import *
def p(g):
 h,w=len(g),len(g[0])
 f=[i for s in g for i in s]
 C=Counter(f).most_common(10)
 C=[c for c in C if c[0]>0]
 g=[[0 for _ in R] for R in g]
 for i in range(len(C)):
  g[-(i+1)]=g[-(i+1)][:w-C[i][1]] + [C[i][0] for _ in range(C[i][1])]
 return g""",
    307: """def p(g):
 import numpy as np
 return np.kron(g, np.ones((2, 2))).tolist()""",
    309: """def p(g):return [[5 if x == 7 else x for x in row] for row in g]""",
    311: """def p(g):return [row + row[::-1] for row in g]""",
    312: """def p(m):
 for r in m:
  for v in r:
   if v and v-5:
    r[:]=[v*(x==5)+x*(x!=5)for x in r];break
 return m""",
    316: """def p(m):
 n = 3
 v = []
 for c in range(len(m[0])):
  for r in range(len(m)):
   if m[r][c]:
    v.append(m[r][c])
    break
 v.extend([0]*(n*n-len(v)))
 return [v[i*n:(i+1)*n] if i%2==0 else v[i*n:(i+1)*n][::-1] for i in range(n)]""",
    317: """def p(grid):
 n = len(grid)
 output = [[0 for _ in range(n)] for _ in range(n)]
 for i in range(n):
  for j in range(n):
   if grid[i][j] == 5:
    for x in range(max(0, i-1), min(n, i+2)):
     for y in range(max(0, j-1), min(n, j+2)):
      output[x][y] = 1
 return output""",
    318: """def p(g):
 for i in range(4):
  for j in range(4):
   g[i][j]+=g[i+5][j]
 g=[[3 if C>0 else 0 for C in R] for R in g]
 g=g[:4]
 return g""",
    320: """def p(g):
 h,w=len(g),len(g[0])
 g=[[2 if C>0 else 0 for C in R] for R in g]
 for c in range(w):
  C=[g[r][c] for r in range(h)][::-1]
  r=sum(C)//2//2
  for i in range(r):
   g[-(i+1)][c]=8
 return g""",
    321: """def p(g):
 for r in range(4):
  for c in range(4):
   if g[r][c+5]>0:
    g[r][c+10]=g[r][c+5]
   if g[r][c]>0:
    g[r][c+10]=g[r][c]
 g=[R[10:] for R in g]
 return g""",
    322: """def p(m):
 for c in range(len(m[0])):
  for r in range(len(m)):
   if m[r][c]:break
  else:continue
  for i in range(r,len(m)):m[i][c]=m[r][c]
 return m""",
    323: """def p(m):
 r,c=len(m),len(m[0])
 o=[i[:]for i in m]
 i,j=[(i,j)for i in range(r)for j in range(c)if m[i][j]][0]
 for a,b in(((-1,1),(2,2)),((1,-1),(2,2))):
  x,y=i,j
  while 1:
   for _ in[0]*b[0]:
    x+=a[0]
    if 0<=x<r:o[x][y]=5
    else:break
   else:
    for _ in[0]*b[1]:
     y+=a[1]
     if 0<=y<c:o[x][y]=5
     else:break
    else:continue
   break
 return o""",
    325: """def p(m):
 r,c,n=len(m),len(m[0]),0
 v=[[0]*c for _ in m]
 def d(y,x):
  s=[(y,x)]
  v[y][x]=1
  while s:
   y,x=s.pop()
   for dy,dx in[(-1,0),(1,0),(0,-1),(0,1)]:
    ny,nx=y+dy,x+dx
    if 0<=ny<r and 0<=nx<c and m[ny][nx] and not v[ny][nx]:
     v[ny][nx]=1
     s+=[(ny,nx)]
 for y in range(r):
  for x in range(c):
   if m[y][x]and not v[y][x]:
    n+=1;d(y,x)
 return[[8*(i==j)for j in range(n)]for i in range(n)]""",
    326: """def p(g):return[r[:2]for r in g[:2]]""",
    327: """def p(g):
 P=[]
 E=enumerate
 X=[[0,0,0,0,0,0] for _ in range(6)]
 for r,R in E(g):
  for c,C in E(R):
   if g[r][c]>0:
    P.append([r,c])
  for p_ in P:
   for i in range(10):
    try:X[p_[0]+i][p_[1]+i]=g[p_[0]][p_[1]]
    except:pass
 return X""",
    329: """def p(g):
 m=len(g[0])//2
 X=[[0 for i in R] for R in g]
 for r in range(len(g)):
  X[r][m]=g[r][m]
 return X""",
    331: """def p(g):
 P=[]
 E=enumerate
 for r,R in E(g):
  for c,C in E(R):
   if g[r][c]==1:
    P.append([r,c])
 for p_ in P:
  p1,p2=p_
  if p1>0:g[p1-1][p2]=2
  if p1<9:g[p1+1][p2]=8
  if p2>0:g[p1][p2-1]=7
  if p2<9:g[p1][p2+1]=6
 return g""",
    334: """def p(g):
 d={2:[[5,5,5],[0,5,0],[0,5,0]],1:[[0,5,0],[5,5,5],[0,5,0]],3:[[0,0,5],[0,0,5],[5,5,5]]}
 f=[i for s in g for i in s]
 return d[max(f)]""",
    337: """def p(g):return [[5 if x==8 else 8 if x==5 else x for x in r] for r in g]""",
    339: """def p(g):
 g=[i for s in g for i in s]
 return [[i for i in g if i>0]]""",
    345: """def p(g):
 h,w=len(g),len(g[0])
 for c in range(w):
  if g[-1][c]==2:
   j=0
   for i in range(h):
    if g[-(i+1)][c+j]==5:
     j+=1
     g[-(i)][c+j]=2
    g[-(i+1)][c+j]=2
 return g""",
    346: """from collections import *
def p(g):
 h,w=len(g),len(g[0])
 for r in range(0,h-3+1,1):
  for c in range(0, w-3+1,1):
   P=g[r:r+3]
   P=[R[c:c+3] for R in P]
   f=[i for s in P for i in s]
   C=Counter(f).most_common(1)
   if min(f)>0 and C[0][1]==8:
    return [[P[1][1]]]""",
    347: """def p(g):
 for r in range(3):
  for c in range(3):
   g[r][c]+=g[r][c+3]
   if g[r][c]>0: g[r][c]=6
 g=[R[:3] for R in g]
 return g""",
    352: """def p(g):
 E=enumerate
 X=[[C for i,C in E(R)] for R in g]
 for r,R in E(g):
  for c,C in E(R):
   if C==2:
    for i in range(-1,2):
     for j in range(-1,2):
      try:
       if [i,j]!=[0,0] and r+i>-1 and c+j>-1:X[r+i][c+j]=1
      except:pass
 return X""",
    356: """def p(g):
 res=[r[:]for r in g]
 for c in range(1,10):
  pos=[(i,j)for i in range(len(g))for j in range(len(g[0]))if g[i][j]==c]
  for i in range(len(pos)):
   for j in range(i+1,len(pos)):
    y1,x1=pos[i]
    y2,x2=pos[j]
    if y1==y2:
     for x in range(min(x1,x2),max(x1,x2)+1):
      res[y1][x]=c
    elif x1==x2:
     for y in range(min(y1,y2),max(y1,y2)+1):
      res[y][x1]=c
 return res""",
    360: """def p(g):
 w=len(g[0])
 sep_c=next(c for c in range(w)if all(g[r][c]==g[0][c]for r in range(len(g)))and g[0][c]!=0)
 left=[row[:sep_c]for row in g]
 right_flipped=[row[sep_c+1:][::-1]for row in g]
 return[[(l if l!=0 else r)for l, r in zip(l_row, r_row)]
  for l_row, r_row in zip(left, right_flipped)]""",
    373: """p=lambda g,h=2,w=6:[[g[h-1-i][j]if j%2 else g[i][j]for j in range(w)]for i in range(h)]""",
    376: """def p(g):
 f=[g[-i] for i in range(2,len(g))]
 g=g+f+g+f+[g[0]]
 return g""",
    380: """def p(g):return[list(r)for r in zip(*g)][::-1]""",
    385: """def p(g):
 h=len(g)
 E=enumerate
 for r,R in E(g):
  for c,C in E(R):
   if r<h//2:
    g[r][c]=g[-(r+1)][c]
 return g""",
    386: """def p(g):
 for r in range(4):
  for c in range(3):
   g[r][c]+=g[r][c+4]
   if g[r][c]>0:g[r][c]=0
   else:g[r][c]=3
 g=[R[:3] for R in g]
 return g""",
    389: """def p(g):
 f=[i for s in g for i in s]
 f=[c for c in set(f) if c not in [0,5]][0]
 g=[[f if C==5 else 0 for C in R] for R in g]
 return g""",
    393: """from collections import *
def p(g):
 h,w=len(g),len(g[0])
 f=[i for s in g for i in s]
 C=Counter(f).most_common(4)
 C=[c for c in C if c[0]>0]
 g=[[c[0]] for c in C]
 return g""",
    395: """def p(g):
 for r in range(3):
  for c in range(3):
   g[r][c]+=g[r+3][c]
   if g[r][c]>0:g[r][c]=0
   else:g[r][c]=2
 return g[:3]""",
    399: """def p(g):
 d={1:[[1,0,0],[0,0,0],[0,0,0]],2:[[1,0,1],[0,0,0],[0,0,0]],3:[[1,0,1],[0,1,0],[0,0,0]],4:[[1,0,1],[0,1,0],[1,0,0]],5:[[1,0,1],[0,1,0],[1,0,1]]}
 X=0
 h,w=len(g),len(g[0])
 for r in range(0,h-2+1,1):
  for c in range(0,w-2+1,1):
   P=g[r:r+2]
   P=[R[c:c+2] for R in P]
   f=[i for s in P for i in s]
   if min(f)>0:
    X+=1
 return d[X]""",
    83: """def p(g):
 t=[r+r[::-1]for r in g]
 b=[r+r[::-1]for r in g[::-1]]
 return t+b""",
    
    91: """def p(g):
 E=enumerate
 Z=[[C for i,C in E(R)] for R in g]
 T,X=[],[]
 for r in range(len(g)):
  for c in range(len(g[0])):
   try:
    if g[r-1][c]==5 and g[r][c]==8:Z[r][c]=5
   except:pass
   try:
    if g[r+1][c]==5 and g[r][c]==8:Z[r][c]=5
   except:pass
 for i in range(len(Z)):
  if 5 in Z[i]:X.append(g[i])
 for R in X:
  for i in range(len(R)):
     if R[i]==5:T.append(i)
 X=[R[min(T):max(T)+1] for R in X]
 return X""",
    
    106: """def p(g):
 X=[]
 r1=[list(r) for r in zip(*g[::-1])]
 r2=[list(r) for r in zip(*r1[::-1])]
 r3=[list(r) for r in zip(*r2[::-1])]
 for r in range(len(g)):
  X.append(g[r]+r1[r])
 for r in range(len(g)):
  X.append(r3[r]+r2[r])
 return X"""
}

In [ ]:
def generate_general_fallback_solution(task_data):
    """
    Try simple general transformations and see if any solve the task.
    """
    from copy import deepcopy

    def all_match(task, func):
        for pair in task["train"] + task["test"] + task.get("arc-gen", []):
            try:
                output = func(pair["input"])
                if output != pair["output"]:
                    return False
            except:
                return False
        return True

    candidates = [
        ("identity", lambda g: g),
        ("transpose", lambda g: [list(row) for row in zip(*g)]),
        ("hflip", lambda g: [row[::-1] for row in g]),
        ("vflip", lambda g: g[::-1]),
        ("rotate180", lambda g: [row[::-1] for row in g[::-1]])
    ]

    for name, fn in candidates:
        if all_match(task_data, fn):
            print_rich(f"[green]Task solved by general fallback: {name}[/green]")
            # generate code string for fn
            if name == "identity":
                return "def p(g): return g"
            elif name == "transpose":
                return "def p(g): return [list(r) for r in zip(*g)]"
            elif name == "hflip":
                return "def p(g): return [r[::-1] for r in g]"
            elif name == "vflip":
                return "def p(g): return g[::-1]"
            elif name == "rotate180":
                return "def p(g): return [r[::-1] for r in g[::-1]]"

    return None



In [ ]:
import os
import json
import zipfile
import copy
import re
import math
import ast
import string
from collections import Counter
from functools import reduce
from typing import List, Tuple

from tqdm import tqdm
from rich import print as print_rich

import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

DIR = "/kaggle"


def check_solution(solution, task_data):
    try:
        namespace = {}
        exec(solution, namespace)
        if 'p' not in namespace:
            return False
        all_examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        for example in all_examples:
            input_grid = copy.deepcopy(example['input'])
            expected = example['output']
            try:
                actual = namespace['p'](input_grid)
                if actual != expected:
                    return False
            except Exception:
                return False
        return True
    except Exception:
        return False

def get_bytes(s):
    return len(s.encode('utf-8'))

def calculate_score(s):
    return max(1, 2500 - get_bytes(s))

def remove_spaces(s):
    """
    Remove unnecessary before/after spaces some symbols / operators
    """
    for c in ['[', ']', '(', ')', '{', '}', '=', '!', '<', '>', '+', '-', '*', '/', '%', ';', ',']:
        s = s.replace(' ' + c, c)
        s = s.replace(c + ' ', c)
    return s

def minimize_indentation(s):
    """
    Minimize indentation to multipliers of 1 instead of usual 4
    """
    leading_spaces = [
        len(m.group(1))
        for m in re.finditer(r'^( +)(?=\S)', s, flags=re.MULTILINE)
    ]
    if not leading_spaces:
        return s

    unit = reduce(math.gcd, leading_spaces)
    if unit <= 1:
        return s

    def _shrink(match: re.Match):
        count = len(match.group(1))
        return ' ' * (count // unit)

    return re.sub(r'^( +)', _shrink, s, flags=re.MULTILINE)

def find_local_variables(s: str):
    """
    Find all variable names inside function
    """
    tree = ast.parse(s)
    local_vars = set()

    class LocalVarVisitor(ast.NodeVisitor):
        def visit_FunctionDef(self, node: ast.FunctionDef):
            for arg in node.args.args:
                local_vars.add(arg.arg)
            self.generic_visit(node)

        def visit_Name(self, node: ast.Name):
            if isinstance(node.ctx, ast.Store):
                local_vars.add(node.id)
            self.generic_visit(node)

    LocalVarVisitor().visit(tree)
    return local_vars

def find_single_letter_variables(s: str):
    """
    Find single letter variable names that are currently used
    """
    return {var for var in find_local_variables(s) if len(var) == 1}

def find_available_single_letter_variables(s: str):
    """
    Find single letter variable names that can be used to replace longer names
    """
    used = find_single_letter_variables(s)
    possible = set(string.ascii_lowercase + string.ascii_uppercase + '_')
    return possible - used

def substitute_range(s):
    """
    If there are at least 3x range() used in function assign it to single letter
    function parameter e.g def p(g,r=range)
    """
    v = find_available_single_letter_variables(s).pop()
    if s.count('range(') >= 3:
        s = s.replace('range', v)
        index = s.find(')')
        return s[:index] + ',' + v + '=range' + s[index:]
    return s

def substitute_enumerate(s):
    """
    If there are at least 2x enumerate() used in function assign it to single letter
    function parameter e.g def p(g,e=enumerate)
    """
    v = find_available_single_letter_variables(s).pop()
    if s.count('enumerate(') >= 2:
        s = s.replace('enumerate', v)
        index = s.find(')')
        return s[:index] + ',' + v + '=enumerate' + s[index:]
    return s

def strip_trailing_whitespaces(s):
    """
    Remove any trailing whitespaces (especially new line after last line of code)
    """
    return s.strip()

def _replace_variable_name(s, old_name, new_name):
    """
    Replace long (2+) variable name with single char variable name
    """
    pattern = rf'\b{re.escape(old_name)}\b'
    return re.sub(pattern, new_name, s)

def shorten_variable_names(s):
    """
    Find all variable that have too long names, find new name for them
    and then replace
    """
    long_variable_names = [x for x in find_local_variables(s) if len(x) > 1]
    available = find_available_single_letter_variables(s)
    for x in long_variable_names:
        new_name = available.pop()
        s = _replace_variable_name(s, x, new_name)
    return s

def join_block_lines(code: str) -> str:
    """
    Join lines when possible

    if a:
        print(1)
    ->
    if a:print(1)

    if a:
        print(1)
        print(2)
        print(3)
    ->
    if a:print(1);print(2);print(3)

    Make sure to not break the code, which is easy to do with multiple indentations or
    if else, or combinations of for, while, if, with inside block
    """
    try:
        tree = ast.parse(code)
    except SyntaxError:
        return code

    for node in ast.walk(tree):
        if isinstance(node, (ast.Module, ast.FunctionDef, ast.AsyncFunctionDef)):
            if (node.body
                and isinstance(node.body[0], ast.Expr)
                and isinstance(node.body[0].value, ast.Constant)
                and isinstance(node.body[0].value.value, str)):
                node.body.pop(0)

    lines = code.splitlines(keepends=True)
    transforms: List[Tuple[int, int, int]] = []

    SIMPLE = (
        ast.Assign, ast.AugAssign, ast.Expr, ast.Return,
        ast.Delete, ast.Pass, ast.Continue, ast.Break,
        ast.Assert, ast.Raise, ast.Global, ast.Nonlocal,
        ast.Import, ast.ImportFrom
    )

    for node in ast.walk(tree):
        if not isinstance(node, (ast.For, ast.While, ast.With,
                               ast.If, ast.FunctionDef)):
            continue
        if isinstance(node, ast.If) and node.orelse:
            continue
        body = getattr(node, 'body', None)
        if not body:
            continue

        if not all(isinstance(stmt, SIMPLE) and stmt.lineno == stmt.end_lineno
                   for stmt in body):
            continue
        transforms.append((node.lineno, body[0].lineno, body[-1].end_lineno))

    transforms.sort(key=lambda t: t[0], reverse=True)
    for hdr, start, end in transforms:
        header_index = hdr - 1
        start_index = start - 1
        end_index = end - 1
        if not (0 <= header_index < len(lines) and start_index <= end_index):
            continue
        header = lines[header_index].rstrip('\n')
        if not header.strip().endswith(':'):
            continue

        parts = [lines[i].strip() for i in range(start_index, end_index+1)]
        joined = ';'.join(parts)
        lines[header_index] = f"{header}{joined}\n"
        del lines[start_index:end_index+1]

    return ''.join(lines)

def remove_empty_lines(s):
    """
    Remove empty lines from code
    """
    lines = s.split('\n')
    return '\n'.join([line for line in lines if line.strip()])

def remove_comments(s):
    """
    Remove single line comments from code
    """
    lines = s.split('\n')
    return '\n'.join([line for line in lines if not line.strip().startswith('#')])

def def_to_lambda(s):
    """
    Try to replace def with lambda
    """
    _def = r'^def\s+p\(([A-Za-z])\):return'
    if '\n' not in s and re.match(_def, s):
        s = re.sub(_def, r'p=lambda \1:', s)
    return s

# Your existing submissions
submissions = [
    "/kaggle/input/d/muhammadqasimshabbir/oh-barnacles/submission",
    "/kaggle/input/solved-127-problems-local-pleaseupvote-submission/submission",
    "/kaggle/input/neurips-2025-google-code-golf-championship1",
    "/kaggle/input/liah-submission-gcgc-v2",
    "/kaggle/input/solution-submission-of-publick-noteboook/SolutinSubmission"
]

simple_solution = """def p(g):return g"""

solved = 0
total_score = 0
os.makedirs(f"{DIR}/working/submission", exist_ok=True)
removed_by_method = Counter()

for task_num in tqdm(range(1, 401)):
    task_id = f"{task_num:03d}"
    solutions = []
    task_data_path = f"{DIR}/input/google-code-golf-2025/task{task_id}.json"
    task_data = json.load(open(task_data_path))

    is_solved = False
    
    # First try predefined solutions if available
    if task_num in PREDEFINED_SOLUTIONS:
        solution_code = PREDEFINED_SOLUTIONS[task_num]
        if check_solution(solution_code, task_data):
            solutions.append(solution_code)
            is_solved = True
            print_rich(f"[yellow]{task_id} - solved by predefined solution dict[/yellow]")
    
    # Then try existing submissions
    if not is_solved:
        for submission_path in submissions:
            task_code_path = f"{submission_path}/task{task_id}.py"
            if not os.path.exists(task_code_path):
                continue
            with open(task_code_path, 'r') as f:
                solution_code = f.read()
            if check_solution(solution_code, task_data):
                solutions.append(solution_code)
                is_solved = True
            print_rich(f"[green]{task_id} - solved by sumissions  [/green]")
            
    # If not solved by predefined or existing submissions, try the ARC generator
    if not is_solved and 'arc_generator' in globals():
        try:
            generated_solution = arc_generator.generate_solution(task_data)
            if arc_generator.verify_solution(generated_solution, task_data):
                solutions.append(generated_solution)
                is_solved = True
                print_rich(f"[yellow]{task_id} - solved by generator[/yellow]")
        except Exception as e:
            print_rich(f"[red]Error generating solution for task {task_id}: {str(e)}[/red]")
            pass
            
    # Fallback: try all predefined solutions if none worked
    if not is_solved:
        for key, sol_code in PREDEFINED_SOLUTIONS.items():
            if check_solution(sol_code, task_data):
                        solutions.append(sol_code)
                        is_solved = True
                        print_rich(f"[magenta]{task_id} - solved by global predefined dict (task {key})[/magenta]")
                        break
    if not is_solved:
        general_code = generate_general_fallback_solution(task_data)
        print_rich(f"[red]{task_id} - solved by general fallback[red]")
        if general_code and check_solution(general_code, task_data):
            solutions.append(general_code)
            is_solved = True

    score = 0.001
    best_solution = simple_solution

    if solutions:
        # Apply all minimization techniques to each solution
        minimized_solutions = []
        for solution in solutions:
            for method in [remove_spaces, minimize_indentation,
                         substitute_enumerate, substitute_range,
                         join_block_lines, strip_trailing_whitespaces,
                         remove_empty_lines, remove_comments,
                         shorten_variable_names, def_to_lambda, remove_spaces]:
                b1 = get_bytes(solution)
                new_solution = solution
                try:
                    new_solution = method(solution)
                except Exception as e:
                    print(e)
                b2 = get_bytes(new_solution)

                if check_solution(new_solution, task_data) and b2 < b1:
                    solution = new_solution
                    removed_by_method[method] += (b1 - b2)
            
            minimized_solutions.append(solution)
        
        best_solution = min(minimized_solutions, key=get_bytes)
        score = calculate_score(best_solution)
        if is_solved:
            solved += 1

    total_score += score

    with open(f"{DIR}/working/submission/task{task_id}.py", "w") as f:
        f.write(best_solution)

# Create the submission zip
with zipfile.ZipFile(f"{DIR}/working/submission.zip", "w") as zipf:
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        zipf.write(f"{DIR}/working/submission/task{task_id}.py", 
                  arcname=f"task{task_id}.py")

print_rich(f"[green]Total solved: {solved} / 400[/green]")
print_rich(f"[blue]LB Score: {total_score:.3f}[/blue]")

for k, v in removed_by_method.most_common():
    print(f"{k.__name__:<30}{v:>5}")